<p align="center">
  <img src="header.png" width="100%">
</p>


<div style="text-align: center;">
    <strong style="display: block; margin-bottom: 10px;">Group P</strong> 
    <table style="margin: 0 auto; border-collapse: collapse; border: 1px solid black;">
        <tr>
            <th style="border: 1px solid white; padding: 8px;">Name</th>
            <th style="border: 1px solid white; padding: 8px;">Student ID</th>
        </tr>
        <tr>
            <td style="border: 1px solid white; padding: 8px;">Beatriz Monteiro</td>
            <td style="border: 1px solid white; padding: 8px;">20240591</td>
        </tr>
        <tr>
            <td style="border: 1px solid white; padding: 8px;">Catarina Nunes</td>
            <td style="border: 1px solid white; padding: 8px;">20230083</td>
        </tr>
        <tr>
            <td style="border: 1px solid white; padding: 8px;">Margarida Raposo</td>
            <td style="border: 1px solid white; padding: 8px;">20241020</td>
        </tr>
        <tr>
            <td style="border: 1px solid white; padding: 8px;">Teresa Menezes</td>
            <td style="border: 1px solid white; padding: 8px;">20240333</td>
        </tr>
    </table>
</div>

### 🔗 Table of Contents <a id='table-of-contents'></a>
1. [Introduction](#introduction): Previous Notebook
2. [Business Understanding](#business-understanding): Previous Notebook  
3. [Data Understanding](#data-understanding): Previous Notebook
4. [Data Preparation](#data-preparation): Previous Notebook + In this one we have the preparation pipeline  
5. [Modeling](#modeling): 
6. [Evaluation](#evaluation)
7. [Conclusion](#conclusion) 

---

### <span style="background-color:#000027; padding:5px; border-radius:5px;"> 📌 Introduction <a id='introduction'></a>

This project follows the **CRISP-DM** methodology to conduct a monthly sales forecast of the smart infrastructure business unit of Siemens. 

<b style="background-color:#A9A9A9; padding:5px; border-radius:5px; display: inline-block;">CRISP-DM</b>

<ul style="margin-bottom: 30px;">
    <li><u>Business Understanding</u>: Defining objectives, assessing resources, and project planning.</li>
    <li><u>Data Understanding</u>: Collecting, exploring, and verifying data quality.</li>
    <li><u>Data Preparation</u>: Selecting, cleaning, constructing, integrating, and formatting data to ensure it is ready for analysis.</li>
    <li><u>Modeling</u>: Selecting and applying various modeling techniques while calibrating their parameters to optimal values.</li>
    <li><u>Evaluation</u>: Select the models which are the best performers and evaluate thoroughly if they align with the business objectives. </li>
    <li><u>Deployment</u>: Bridge between data mining goals and the business application of the finalized model.</li>
</ul>

---
<b style="background-color:#A9A9A9; padding:5px; border-radius:5px; display: inline-block;">Montlhy Sales Forecast</b>

<p style="margin-bottom: 50px;"> This case study focuses on ... </p>

---

<b style="background-color:#A9A9A9; padding:5px; border-radius:5px; display: inline-block;">Market Data</b><br>
(https://ec.europa.eu/eurostat/statistics-explained/index.php?title=Thematic_glossaries)<br>
Due to the lack of information about the macroeconomic indicatores used, our analysis was based on the assumption that the following definitions are correct.


- The **Production index** measures changes in value added at factor cost of industry and construction over a month. It does this by measuring changes in the volume of output and activity monthly.
- The **Shipment index** is a measure of the volume of goods shipped by manufacturers monthly. It is an indicator of economic trends, since it reflects production, supply chain dynamics and overall industrial performance. 
- The **World prices of materials** is monthly average global market prices for key industrial inputs, such as energy, heavy materials, etc. These are likely to fluctuate with changes in supply/demand dynamics and economic conditions. 
- The **United States: EUR in LCU** is the monthly exchange rate of the Euro, expressed in local currency units from the United States (US dollar), representing the number of US dollars per euro. It indicates currency fluctuation, which often impacts trade.
- The **Producer prices** measures the monthly change in the trading price of products and related services, from the seller's perspective. It does not only serve as an early indicator of inflationary pressures in the economy before it reaches the consumer, but it can also record the evolution of prices over longer time periods.

<br>

| **Features**                                    | **Feature Description**                                                                                             |
|-------------------------------------------------|---------------------------------------------------------------------------------------------------------------------|
| *MAB_ELE_PRO156*                                | **Production index** for machinery and electrical equipment in China.                                                     |
| *MAB_ELE_SHP156*                                | **Shipments index** for machinery and electrical equipment in China.                                                     |
| *MAB_ELE_PRO250*                                | **Production index** for machinery and electrical equipment in France.                                                    |
| *MAB_ELE_SHP250*                                | **Shipments index** for machinery and electrical equipment in France.                                                    |
| *MAB_ELE_PRO276*                                | **Production index** for machinery and electrical equipment in Germany.                                                  |
| *MAB_ELE_SHP276*                                | **Shipments index** for machinery and electrical equipment in Germany.                                                  |
| *MAB_ELE_PRO380*                                | **Production index** for machinery and electrical equipment in Italy.                                                    |
| *MAB_ELE_SHP380*                                | **Shipments index** for machinery and electrical equipment in Italy.                                                    |
| *MAB_ELE_PRO392*                                | **Production index** for machinery and electrical equipment in Japan.                                                    |
| *MAB_ELE_SHP392*                                | **Shipments index** for machinery and electrical equipment in Japan.                                                    |
| *MAB_ELE_PRO756*                                | **Production index** for machinery and electrical equipment in Switzerland.                                              |
| *MAB_ELE_SHP756*                                | **Shipments index** for machinery and electrical equipment in Switzerland.                                              |
| *MAB_ELE_PRO826*                                | **Production index** for machinery and electrical equipment in United Kingdom.                                           |
| *MAB_ELE_SHP826*                                | **Shipments index** for machinery and electrical equipment in United Kingdom.                                           |
| *MAB_ELE_PRO840*                                | **Production index** for machinery and electrical equipment in the United States.                                        |
| *MAB_ELE_SHP840*                                | **Shipments index** for machinery and electrical equipment in the United States.                                        |
| *MAB_ELE_PRO1100*                               | **Production index** for machinery and electrical equipment in Europe.                                                   |
| *MAB_ELE_SHP1100*                               | **Shipments index** for machinery and electrical equipment in Europe.                                                   |
| *RohiBASEMET1000_org*                           | **World price** of base metals.                                                                                          |
| *RohiENERGY1000_org*                            | **World price** of energy.                                                                                               |
| *RohiMETMIN1000_org*                            | **World price** of metals and minerals.                                                                                   |
| *RohiNATGAS1000_org*                            | **World price** of natural gas index.                                                                                     |
| *RohCRUDE_PETRO1000_org*                        | **World price** of crude oil (average).                                                                                   |
| *RohCOPPER1000_org*                             | **World price** of copper.                                                                                                |
| *WKLWEUR840_org*                                | United States: **Euros in local currency units** (LCU).                                                                    |
| *PRI27840_org*                                  | United States: Electrical equipment (**Producer Prices**).                                                                |
| *PRI27826_org*                                  | United Kingdom: Electrical equipment (**Producer Prices**).                                                               |
| *PRI27380_org*                                  | Italy: Electrical equipment (**Producer Prices**).                                                                        |
| *PRI27250_org*                                  | France: Electrical equipment (**Producer Prices**).                                                                       |
| *PRI27276_org*                                  | Germany: Electrical equipment (**Producer Prices**).                                                                      |
| *PRI27156_org*                                  | China: Electrical equipment (**Producer Prices**).                                                                       |
| *PRO28840_org*                                  | United States: Machinery and equipment n.e.c. (**Production index**).                                                      |
| *PRO281000_org*                                 | World: Machinery and equipment n.e.c. (**Production index**).                                                             |
| *PRO28756_org*                                  | Switzerland: Machinery and equipment n.e.c. (**Production index**).                                                       |
| *PRO28826_org*                                  | United Kingdom: Machinery and equipment n.e.c. (**Production index**).                                                    |
| *PRO28380_org*                                  | Italy: Machinery and equipment n.e.c. (**Production index**).                                                             |
| *PRO28392_org*                                  | Japan: Machinery and equipment n.e.c. (**Production index**).                                                             |
| *PRO28250_org*                                  | France: Machinery and equipment n.e.c. (**Production index**).                                                            |
| *PRO28276_org*                                  | Germany: Machinery and equipment n.e.c. (**Production index**).                                                           |
| *PRO27840_org*                                  | United States: Electrical equipment (**Production index**).                                                               |
| *PRO271000_org*                                 | World: Electrical equipment (**Production index**).                                                                      |
| *PRO27756_org*                                  | Switzerland: Electrical equipment (**Production index**).                                                                 |
| *PRO27826_org*                                  | United Kingdom: Electrical equipment (**Production index**).                                                             |
| *PRO27380_org*                                  | Italy: Electrical equipment (**Production index**).                                                                      |
| *PRO27392_org*                                  | Japan: Electrical equipment (**Production index**).                                                                      |
| *PRO27250_org*                                  | France: Electrical equipment (**Production index**).                                                                     |
| *PRO27276_org*                                  | Germany: Electrical equipment (**Production index**).                                                                    |

<br>

---

<b style="background-color:#A9A9A9; padding:5px; border-radius:5px; display: inline-block;">Sales Data</b>
| Features                                      | Feature Description |
|-----------------------------------------------|---------------------|
| *DATE*                                       | Day at which a sale was made, in the format *dd-mm-yyyy* |
| *Mapped_GCK*                                 | Product group of the product sold |
| *Sales_EUR*                                  | Monetary amount made from the sale, in Euros  |

<br>

---

In [1]:
# pip install prophet == 1.1.6
# pip install keras == 3.9.0 tensorflow == 2.19.0
# pip install neuralprophet == 0.8.0
# pytorch-forecasting-1.3.0
#pip install nixtla == 0.6.6
#pip install pmdarima == 2.0.4
#pip install scikeras == 0.13.0
#pip install keras-tcn == 3.1.2

In [1]:
import os
import pickle
import pandas as pd
import numpy as np

import seaborn as sns

from itertools import combinations
from sklearn.model_selection import PredefinedSplit, GridSearchCV, TimeSeriesSplit
from joblib import Parallel, delayed
from sklearn.preprocessing import StandardScaler


#timesgpt
from nixtla import NixtlaClient
#xgboost
import xgboost as xgb
from xgboost.callback import EarlyStopping
from xgboost import XGBRegressor
#tcn
from scikeras.wrappers import KerasRegressor
from tcn import TCN
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.models import Sequential
#arima
from statsmodels.tsa.arima.model import ARIMA
from pmdarima import auto_arima
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.stattools import adfuller
import warnings
#NeuralProphet
from prophet import Prophet
from neuralprophet import NeuralProphet

#evaluation
from sklearn.metrics import (
    mean_squared_error, 
    r2_score, 
    mean_absolute_percentage_error,
    mean_absolute_error,
    root_mean_squared_error)



ERROR:NP.plotly:Importing plotly failed. Interactive plots will not work.
ERROR:NP.plotly:Importing plotly failed. Interactive plots will not work.


In [2]:
# Set pandas display options
pd.set_option('display.max_columns', None)  # Show all columns
pd.set_option('display.width', 1000)       # Set display width
pd.set_option('display.max_colwidth', 100) # Show full feature lists
pd.set_option('display.float_format', '{:.4f}'.format)  # 4 decimal places

# If you want to force standard notation (no scientific):
pd.set_option('display.float_format', lambda x: '%.4f' % x if abs(x) > 1e-4 else '%.4e' % x)

In [59]:
import pandas as pd
from IPython.display import display, HTML

def display_excel_sheets_side_by_side(excel_path, tablespacing=5, max_height="500px"):
    """
    Displays all sheets from an Excel file as tables in a scrollable horizontal div.
    
    Args:
        excel_path (str): Path to the Excel file.
        tablespacing (int): Spacing between tables (in pixels).
        max_height (str): Maximum height of the table container (e.g., "500px").
    """
    # Read all sheets from the Excel file
    sheets_dict = pd.read_excel(excel_path, sheet_name=None)
    
    # Generate HTML for each sheet
    html = """
    <div style='overflow-x: auto; white-space: nowrap; padding-bottom: 10px; border-bottom: 2px solid #ddd;'>
    """
    
    for sheet_name, df in sheets_dict.items():
        # Convert DataFrame to HTML and style it
        table_html = df.to_html(
            index=False, 
            classes='dataframe', 
            border=0,
            max_rows=10  # Limit rows for display (optional)
        )
        
        # Wrap each table in a scrollable div
        html += f"""
        <div style='display:inline-block; margin-right:{tablespacing * 5}px; vertical-align:top;'>
            <h4 style='text-align:center;'>{sheet_name}</h4>
            <div style='max-height:{max_height}; overflow-y:auto; border:1px solid #ccc; padding:5px;'>
                {table_html}
            </div>
        </div>
        """
    
    html += "</div>"
    display(HTML(html))

### <span style="background-color:#000027; padding:5px; border-radius:5px;"> 📌 Business Understanding <a id='business-understanding'></a>

##### Click [here](#table-of-contents) ⬆️ to return to the Index.
---

The **Business Understanding** phase of the project entails the comprehension of the background leading to the project, as well as the business goals and requirements to be achieved. 
<br><br><br>
Siemens is a global technology company that provides multi-industry solutions, with a strong focus on automation, digitalization, and sustainability. 

**Smart Infrastructure** division at Siemens

(https://www.siemens.com/global/en/company/about/businesses/smart-infrastructure.html
https://www.siemens.com/global/en/company/about/history/company/2007-2018.html)

In October 2018, Siemens launched its Smart Infrastructure division, as part of the implementation of an optimized corporate structure. This new division emerged from the merger of former divisions, specifically the power distribution segment of the Energy Management division, the Building Technologies’ business, and the control products business of the Digital Factory division.
Smart Infrastructure helps customers through their digital transformation journey by leveraging cutting-edge technology that enables clients to adapt to evolving environments and to seize new opportunities. It mostly provides B2B services, focusing on energy efficiency and resource optimization, which in turn improves clients’ asset performance, availability, reliability, and operational efficiency. All in all, this division future-proofs infrastructures, fostering collaborative ecosystems.

<br><br>
**Economic Environment 2018**

(Siemens. (2018). Annual report 2018. Siemens AG. https://www.siemens.com/global/en/company/investor-relations/events-publications-ad-hoc/annualreports.html)

`Overall economic conditions`
In 2018, advanced countries saw GDP growth of 2.3%, mainly due to the U.S. economy benefiting from tax cuts. In contrast, emerging countries' GDP growth fell slightly from 4.7% to 4.6%, affected by capital outflows and currency depreciation, notably in Argentina and Turkey. This decline was worsened by U.S. monetary tightening and fears of a global trade war from new tariffs. Political tensions increased in the Middle East, particularly with U.S. sanctions on Iran. In Europe, uncertainties around the U.K.'s exit from the EU and worries about Italy’s budget led to negative investment sentiment. All in all, the economic acceleration at the beginning of 2018 lost steam towards the end of the year due to these adverse effects.

`Energy Management`
The Energy Management Division offers a wide spectrum of software, products, systems, solutions, and services for transmitting, distributing and managing electrical power and for providing intelligent power infrastructure.
This sector faces competition from large multinational companies and fast-growing firms in emerging countries. It generally benefits from major trends, in particular decarbonization, decentralization and digitalization. Although Energy Management reported a double-digit order decline, due to a sharply lower volume of large orders in the transmission solutions business, revenue grew in most Division’s businesses, led by the power distribution businesses. For the latter, on a geographic basis, higher revenue in the region Asia, Australia was partially offset by a decline in the Americas strongly influenced by currency headwinds, while revenue in the region Europe, C. I. S., Africa, Middle East was flat year-over-year.
Under the new organizational structure, transmission solutions will be assigned to the Operating Company Gas and Power, while the businesses for power distribution will be assigned to the Operating Company Smart Infrastructure.


`Building Technologies`
The Building Technologies Division is a leading provider of automation technologies and digital services for safe, secure and efficient buildings and infrastructures. The Division offers products, solutions, services and software for fire safety, security, building automation and energy management.
The Division competes with multinational companies, system integrators, local firms, and IT startups. Strong competition is causing price pressure, especially in the solutions business. Economic changes affect the Division's activities with delay, and particularly in non-residential construction markets, there is a time lag of two to four quarters. Key trends include the demand for energy efficiency and smart space solutions to improve comfort and productivity. In fiscal 2018, the Building Technologies acquired three startups to strengthen its portfolio and expertise. Revenue increased, with the Division’s revenue growing faster than the market on increases in all reporting regions, with strongest market growth contributions came from the Asia, Australia region, particularly including India. Orders grew in the region Europe, C. I. S., Africa, Middle East, driven by double-digit growth in Germany including multiple multi-year service contracts. The Division won major orders in the U. S. in both periods under review. Profit and profitability remained strong, supported by economies of scale and improved productivity. The markets are expected to grow solidly in fiscal 2019, although U.S. growth may slow slightly.


`Digital Factory`
The Digital Factory Division provides a wide range of automation products and solutions for manufacturing, including software, control systems, motors, and integrated systems. It also offers lifecycle services and software for design and testing. Changes in customer demand are strongly driven by macroeconomic cycles, leading to variations in profitability. Competition is mainly from large multinational firms offering a relatively broad portfolio, and companies that are only active in certain geographic or product markets. Key trends include rising interest rates, which will limit investment but at the same time make it more focused on improving efficiency, a shift towards regionalization solutions, either to protect local economies or to better adapt solutions to local needs, and an increased focus on digitalization for competitiveness. The Division reported year-over-year growth in orders and revenue, especially in software. Growth was strongest in the Americas and Asia, notably China, and customer investments grew faster than their production, underlining a clear growth momentum relating to automation and digitalization. While the market is expected to continue growing, slower demand is anticipated for 2019 due to the high investment levels of 2018 and rising global trade tensions.

<br><br>
**Economic Environment 2019**

(Siemens. (2019). Annual report 2019. Siemens AG. https://www.siemens.com/global/en/company/investor-relations/events-publications-ad-hoc/annualreports.html)

`Overall economic conditions`
After GDP growth of 3.2% in 2018, it is expected to drop to 2.6% in 2019, as to the U.S.-China trade dispute and geopolitical tensions in the Middle East contributed to uncertainty, which weighed on capital investments. Global exchange of goods started to decline from October 2018, leading to near-stagnation of industrial production afterwards. The decline affected regions reliant on trade and industry, particularly Germany and Italy in Europe, Mexico and Canada in the Americas, and India in Asia. In addition, European economies suffered from continued uncertainties regarding the U. K. leaving the European Union (Brexit) and the budget clash between the European Commission and Italy’s government, both weighing especially on business investment environment. Countries dependent on commodities and raw material exporting, notably Chile, Brazil and Argentina, saw declines in commodity prices in addition to other adverse factors including domestic political and financial instability. Advanced countries are expected to grow by 1.6%, 0.7 percentage points lower than 2018, while emerging countries will see a decline from 4.6% to 4.1%.

`Smart Infrastructure`
Smart Infrastructure brings together energy supply – from intelligent control across the grid and low- and medium-voltage electrification and control products – with building technology, targeting high-growth areas like energy storage, electric vehicle infrastructure, and microgrids at the grid edge. It reaches customers through multiple channels, including global sales, distributors, OEMs, resellers, installers, and direct sales via regional branch offices. Its diverse customer base encompasses infrastructure developers, construction firms, public and commercial buildings, utilities, and heavy industries. Smart Infrastructure faces competition from large multinationals, smaller manufacturers, local integrators, and facility management firms. Economic changes impact customer demand differently, with discrete manufacturing reacting quickly and strongly with macroeconomic cycles, while other sectors - infrastructure, construction, heavy industries and the utilities – have a slower response to economic cycles. Key trends include rising population and urbanization, the need for safe and sustainable environments, and decarbonization, growing decentralization and prosumers’ influence. Orders and revenue increased across all business areas and regions in fiscal 2019.  

<br><br>
**Economic Environment 2020**

(Siemens. (2020). Annual report 2020. Siemens AG. https://www.siemens.com/global/en/company/investor-relations/events-publications-ad-hoc/annualreports.html)

`Overall economic conditions`
The fiscal year 2020 was heavily affected by the coronavirus pandemic and subsequent recession, the deepest since World War II. Global GDP is expected to fall by 4.5% in 2020, following a growth of 2.6% in 2019. Economic activity was already decelerating due to the U.S.-China trade conflict, but economic sentiment indicators were starting to improve in response to calming of the conflict ( “phase one deal”). Then COVID-19 emerged and started to spread globally, resulting in strict social distancing measures that limited operations, especially in contact-heavy sectors. Many other industries were directly affected by supply chain problems or indirectly by insufficient demand and stopped production.

Governments and central banks responded with strong fiscal and monetary policies to support businesses and households, leading to a brief economic rebound in the summer. However, renewed virus outbreaks hindered full recovery and restrictions in high contact industries made the economy operate only at about 90% of its full potential, with advanced economies projected to see a 5.5% decline in GDP. Siemens experienced mixed impacts across its sectors, benefiting from growth in technology-related areas, while facing challenges in customer access and sales. While the pandemic significantly slowed our sales and service activities, this also resulted in cost reductions such as lower travel and marketing expenses. Furthermore, we were able to keep our production largely stable due to the use of our own technology in our factories and our diversified value chain.


`Smart Infrastructure`
Smart Infrastructure’s businesses are impacted by changes in the overall economic environment to varying degrees, depending on customer segment. Particularly in its solutions and service business, it is affected by changes in the non-residential building construction markets with a time lag of two to four quarters.
Smart Infrastructure is experiencing a decline in orders, particularly in the solutions and services business, which previously saw a higher volume of orders. The products business showed only a slight decrease, while orders in the systems and software business stayed stable. Overall, revenue for the solutions and services business and for the systems and software business remained close to the prior- year levels. The drop in orders and revenue primarily came from Europe, Africa, the Middle East, and parts of Asia and Australia, but the Americas remained stable.
Demand in Smart Infrastructure markets declined moderately due to COVID-19, especially in the automotive, oil and gas, and machine-building sectors. However, energy performance services and the data center market grew due to the increased focus on energy efficiency, digital services and remote work. 
Market development will be affected by lower demand from building construction, leading to a slight decline in volume of markets served by Smart Infrastructure year-over-year. Asia and Australia are expected return to normal growth, while Europe and the U.S. may see declines.

<br><br>
**Economic Environment 2021**

(Siemens. (2021). Annual report 2021. Siemens AG. https://www.siemens.com/global/en/company/investor-relations/events-publications-ad-hoc/annualreports.html)

`Overall economic conditions`
In fiscal 2021, the global economy was heavily influenced by the COVID-19 pandemic and its many repercussions. After a GDP contraction in 2020, a strong recovery is expected in 2021. The economy grew rapidly in the third quarter of 2020 after the first wave of COVID-19. Despite fears of a new recession due to subsequent COVID-19 waves, economic activity had already adapted, supported by significant stimulus measures in Europe and the U.S.
Central banks gave support with expansionary measures, while short-term interest rates were at or near zero. Accordingly, the global economy continued to expand also in the fourth quarter of calendar 2020 and the first quarter of calendar 2021, despite renewed outbreaks and lockdowns. In December 2020, the first countries approved new COVID-19 vaccines.
However, momentum weakened in early 2021 due to rising infections and the more contagious Delta variant. Vaccination efforts lagged, especially in emerging countries, while supply disruptions affected many sectors, leading to logistics bottlenecks that disrupted value chains from raw materials to high-tech goods, especially semiconductors, and caused extraordinary disruptions in global logistics systems. High demand for goods, driven by consumer savings, contributed to increased producer and energy prices,  resulting in elevated rates of inflation. 
The Chinese economy – with the world’s largest manufacturing sector –  particularly benefited from global demand. However, tensions in the property sector and energy shortages weighed on economic activity in the second half of calendar 2021. 
Overall, major economies experienced strong rebounds, and GDP is expected to grow strongly.

`Smart Infrastructure`
Smart Infrastructure is benefitting from several positive trends, including urbanization, demographic changes, climate change, and digitalization. Urbanization and demographic shifts create a demand for smarter buildings, while climate change pushes for decarbonization and the development of flexible energy infrastructures. Digitalization enables the improvement of building efficiency and electricity management with more renewable energy sources.
Orders for Smart Infrastructure improved across all sectors, with notable growth in the products and systems businesses, driven by demand from industrial customers and contracts from U.S. semiconductor manufacturers. Revenue also grew, although the solutions and services business saw a slight decline due to negative currency translation effects. Meanwhile, supply chain challenges were managed effectively, preventing major disruptions.
Overall, markets served by Smart Infrastructure grew moderately in fiscal 2021, experiencing a recovery from COVID-19-related effects that had a strong impact on most customer industries a year earlier.
Smart Infrastructure also experienced a number of supply chain constraints, especially in the areas of base metals (copper, aluminum, steel), plastics, semiconductors and transportation services. Whereas the management of these constraints required additional effort, Smart Infrastructure’s supply chains have proven to be resilient, so that major interruptions could be avoided, and delivery ability was maintained.
Market growth is expected to accelerate, primarily fueled by demand in the pharmaceutical sector, data centers, and utilities. Growth will be strongest in Asia and Australia. Growth in the region Europe, C.I.S., Africa, and the Middle East is expected to accelerate and markets in the region Americas are expected to return to growth.

<br><br>
**Economic Environment 2022**

(Siemens. (2022). Annual report 2022. Siemens AG. https://www.siemens.com/global/en/company/investor-relations/events-publications-ad-hoc/annualreports.html)

`Overall economic conditions`
In fiscal 2022, global economic growth faced major disruptions from the war in Ukraine, the effects of COVID-19, and the economic slowdown in China. After a strong recovery in 2021, GDP growth is expected to decline as the economic rebound ended abruptly. In late 2021, increased vaccinations and reduced restrictions boosted consumer spending, triggering inflationary pressures, especially in the U.S. and Europe. Supply chain issues, labor shortages, and heightened energy prices contributed to this inflation, while large stimulus packages and high household savings fueled it. 
The war in Ukraine further strained economies in early 2022, causing another significant rise in energy prices, which heavily impacted Europe and industrial sectors, especially energy-intensive industries such as chemicals. The war in Ukraine put further pressure on developing economies, especially in the Middle East, Africa, and Turkey, as both Russia and Ukraine were major exporters of grain and fertilizer before the war.
China’s zero-COVID strategy became even more strict with the emergence of the Delta variant and the highly infectious Omicron variant, which hindered economic activity, along with regulations affecting growing sectors and a recession in real estate. Consequently, China's growth was predicted to slow. 
In response to rising inflation, the U.S. Federal Reserve raised interest rates and tightened monetary policy, followed by other central banks. This led to economic disruptions, resulting in slower-than-expected GDP growth for 2022.

`Smart Infrastructure`
In fiscal 2022, Smart Infrastructure acquired Brightly Software Inc., improving its position in software for asset and maintenance management. Smart Infrastructure’s businesses are impacted by changes in the overall economic environment to varying degrees. Especially, demand for service offerings shows only limited influence from macroeconomic cycles.
Orders and revenue grew significantly across all businesses, especially in electrical products and electrification, driven by industrial customers, data centers, and proactive purchasing. Smart Infrastructure faced supply chain challenges but avoided major disruptions. The Americas region saw the strongest growth, while Asia and Australia were hindered by COVID-19 impacts, particularly in China. Both order and revenue development included positive currency translation effects. 
Overall, markets served by Smart Infrastructure grew clearly in fiscal 2022. Market growth is anticipated to be slightly slower due to ongoing inflation and supply chain issues, with solid demand expected in data centers and power distribution despite challenges like price increases and geopolitical tensions.
Market development is expected to continue to be influenced by supply chain constraints and effects from the war in Ukraine, including on energy prices. Further impacts could arise from potential lockdown measures in China and geopolitical tensions.

### <span style="background-color:#000027; padding:5px; border-radius:5px;"> 📌 Modeling <a id='modeling'></a>

[1. Data Loading](#data-loading-and-description)

[2. Usefull Functions](#functions)

[3. Model Selection per Product](#model-selection)
- [3.1. TimeGPT](#timegpt)
- [3.2. XGBoost](#xgboost)
- [3.3. TCN](#tcn) 
- [3.4. Prophet](#prophet) 
- [3.5. ARIMA](#arima)
- [3.6. Results Comparison](#compare) 

[4. Exogenous Variables Prediction](#market)

[5. Final Modeling](#final)


Click [here](#table-of-contents) ⬆️ to return to the Index.

---

#### <span style="background-color:#000027; padding:5px; border-radius:5px;">**1. Data Loading**</span> <a id='data-loading-and-description'></a>  

Click [here](#table-of-contents) ⬆️ to return to the Index.


In [3]:
def load_dfs_from_folder(folder_path):
    """Loads DataFrames from files in a specified folder and returns a dictionary."""
    dfs = {}
    # List all files in the folder
    for file_name in os.listdir(folder_path):
        if file_name.endswith(".pkl"):
            key = file_name.replace(".pkl", "")  # Extract key from the file name
            file_path = os.path.join(folder_path, file_name)
            
            # Load the dataframe from the pickle file
            with open(file_path, 'rb') as f:
                dfs[key] = pickle.load(f)
            print(f"Loaded {key} from {file_path}")
    
    return dfs

# Load both product_dfs and lagged_product_dfs from their respective folders
product_dfs = load_dfs_from_folder("product_dfs_folder")
lagged_product_dfs = load_dfs_from_folder("lagged_product_dfs_folder")

Loaded P1 from product_dfs_folder\P1.pkl
Loaded P11 from product_dfs_folder\P11.pkl
Loaded P12 from product_dfs_folder\P12.pkl
Loaded P13 from product_dfs_folder\P13.pkl
Loaded P14 from product_dfs_folder\P14.pkl
Loaded P16 from product_dfs_folder\P16.pkl
Loaded P20 from product_dfs_folder\P20.pkl
Loaded P3 from product_dfs_folder\P3.pkl
Loaded P36 from product_dfs_folder\P36.pkl
Loaded P4 from product_dfs_folder\P4.pkl
Loaded P5 from product_dfs_folder\P5.pkl
Loaded P6 from product_dfs_folder\P6.pkl
Loaded P8 from product_dfs_folder\P8.pkl
Loaded P9 from product_dfs_folder\P9.pkl
Loaded Sales_CPI from product_dfs_folder\Sales_CPI.pkl
Loaded P1 from lagged_product_dfs_folder\P1.pkl
Loaded P11 from lagged_product_dfs_folder\P11.pkl
Loaded P12 from lagged_product_dfs_folder\P12.pkl
Loaded P13 from lagged_product_dfs_folder\P13.pkl
Loaded P16 from lagged_product_dfs_folder\P16.pkl
Loaded P20 from lagged_product_dfs_folder\P20.pkl
Loaded P3 from lagged_product_dfs_folder\P3.pkl
Loaded P36 

In [4]:
for product_id in product_dfs.keys():
    product_dfs[product_id] = product_dfs[product_id].rename(columns={product_id: "Sales"})

for product_id in lagged_product_dfs.keys():
    lagged_product_dfs[product_id] = lagged_product_dfs[product_id].rename(columns={product_id: "Sales"})

In [5]:
product_dfs['P1'].head()

,Sales,MAB_ELE_SHP840,PRI27276_org,PRO27826_org,MAB_ELE_PRO276,MAB_ELE_SHP1100
month_year,,,,,,
2018-10-01,35774028.5209,127.8088,109.1196,118.6708,124.2279,130.9893
2018-11-01,5063648.6000,117.6759,109.2248,120.4670,127.4041,132.9341
2018-12-01,37321267.9382,123.2801,109.3301,105.3787,120.5186,131.2613
2019-01-01,27090400.9380,111.0438,109.7510,107.1749,104.7763,113.0576
2019-02-01,34132093.4229,116.7369,109.8562,110.6476,109.5970,117.7047


In [6]:
lagged_product_dfs['P8'].head()

,Sales,PRI27840_org,RohCOPPER1000_org,time_idx,year,month,P8_lag_1,P8_lag_2,P8_lag_5,P8_lag_6,P8_lag_10,P8_ma_1,P8_ma_2,P8_ma_5,P8_ma_6,P8_ma_10
month_year,,,,,,,,,,,,,,,,
2019-08-01,398332.2744,110.6561,75.7745,10,2019,8,368081.2008,420287.0223,582419.0346,361474.5342,580778.2653,398332.2744,383206.7376,380139.7221,413852.9409,400303.6854
2019-09-01,1086855.2918,111.0503,76.4355,11,2019,9,398332.2744,368081.2008,416386.5006,582419.0346,518398.3785,1086855.2918,742593.7831,514233.4804,497925.6504,457149.3767
2019-10-01,112014.9227,111.0322,76.4097,12,2019,10,1086855.2918,398332.2744,297611.6126,416386.5006,267418.3494,112014.9227,599435.1073,477114.1424,447197.0541,441609.0341
2019-11-01,540208.8029,111.0354,77.7720,13,2019,11,112014.9227,1086855.2918,420287.0223,297611.6126,372627.9465,540208.8029,326111.8628,501098.4985,487629.9191,458367.1197
2019-12-01,491332.7200,111.2866,80.6535,14,2019,12,540208.8029,112014.9227,368081.2008,420287.0223,361474.5342,491332.7200,515770.7614,525748.8024,499470.8688,471352.9383


#### <span style="background-color:#000027; padding:5px; border-radius:5px;">**2. Usefull Functions**</span> <a id='functions'></a>  

Click [here](#table-of-contents) ⬆️ to return to the Index.


In [7]:
def preprocessing_pipeline(train, val=None, test=None, outlier_treatment=True):

    def winsorize_function(df, cols, lower_quantile=0.01, upper_quantile=0.99):
        """Apply Winsorization only to the product sales column"""
        df = df.copy()
        bounds = {}
        for col in cols:
            q1 = df[col].quantile(lower_quantile)
            q3 = df[col].quantile(upper_quantile)
            df[col] = df[col].clip(lower=q1, upper=q3)
            bounds[col] = (q1, q3)  # Store actual percentile values
            print(f"{col}: Winsorized Bounds -> Lower = {q1:.2f}, Upper = {q3:.2f}")

        return df, bounds
        
    def process_dataset(df, cols, is_train=True, bounds=None):
        df = df.copy() 
        if is_train:
            if outlier_treatment:
                df, bounds = winsorize_function(df, cols)
            else:
                bounds = {}  
        else:
            if outlier_treatment:
                if bounds is None:
                    raise ValueError("Bounds must be provided for validation and test datasets.")
                for col in cols: 
                    if col in bounds:
                        lower, upper = bounds[col]
                        df[col] = df[col].clip(lower, upper)  # Corrected clipping
        return (df, bounds) if is_train else df

    # Process the training dataset
    train, bounds = process_dataset(train, cols = train.columns, is_train=True)

    # Process validation and test datasets with correct bounds
    if val is not None:
        val = process_dataset(val, cols = val.columns, is_train=False, bounds=bounds)

    if test is not None:
        test = process_dataset(test, cols = test.columns, is_train=False, bounds=bounds)
        
    # Return the datasets 
    if test is not None and val is not None:
        return train, val, test
    elif val is not None:
        return train, val
    elif test is not None:
        return train, test
    else:
        return train

def time_series_train_test_split(X, y, test_size=10):
    """Split time series data maintaining temporal order"""
    split_idx = len(X) - test_size
    return (
        X.iloc[:split_idx], X.iloc[split_idx:],
        y.iloc[:split_idx], y.iloc[split_idx:])

def prepare_time_series_data(df_train, feature_set, target_col="Sales", horizon=10, winsorize=False, scaling=False):

    # Extract product data
    data = df_train.copy()
    
    y = data[[target_col]]  
    X = data.drop(columns=[target_col])
    
    # Select features - handle empty feature_set case
    if feature_set:
        try:
            # Convert feature_set to list if it's not already
            if not isinstance(feature_set, list):
                feature_set = [feature_set]
            # Select only columns that exist in X
            available_features = [f for f in feature_set if f in X.columns]
            X = X[available_features]
        except KeyError as e:
            raise ValueError(f"Some features not found in DataFrame: {e}")
    
    # Train/Test Split
    X_train, X_val, y_train, y_val = time_series_train_test_split(X, y, test_size=horizon)

    # Apply preprocessing steps
    y_train, y_val = preprocessing_pipeline(y_train, y_val, test=None, outlier_treatment=winsorize)

    # Scaling
    if scaling: 
        scaler_X = StandardScaler()
        X_train_scaled = scaler_X.fit_transform(X_train)
        X_val_scaled = scaler_X.transform(X_val)
        
        scaler_y = StandardScaler()
        y_train_scaled = scaler_y.fit_transform(y_train)
        y_val_scaled = scaler_y.transform(y_val)
        
        # Convert back to DataFrame for easier handling
        X_train_scaled = pd.DataFrame(X_train_scaled, index=X_train.index, columns=X_train.columns)
        X_val_scaled = pd.DataFrame(X_val_scaled, index=X_val.index, columns=X_val.columns)
        y_train_scaled = pd.DataFrame(y_train_scaled, index=y_train.index, columns=y_train.columns)
        y_val_scaled = pd.DataFrame(y_val_scaled, index=y_val.index, columns=y_val.columns)
        
        return X_train_scaled, X_val_scaled, y_train_scaled, y_val_scaled
    else:
        return X_train, X_val, y_train, y_val

In [8]:
def generate_feature_combinations(features, max_features=None):
    """Generate all possible feature combinations"""
    if max_features is None:
        max_features = len(features)
    
    all_combinations = []
    for r in range(1, max_features + 1):
        all_combinations.extend(combinations(features, r))
    
    return [list(comb) for comb in all_combinations]

In [9]:
def convert_results_to_df(results):
    if results is None or len(results) == 0:
        return pd.DataFrame()

    data = []
    for result in results:
        # Assuming each item in the list is a dictionary
        data.append({
            'product_id': result['product_id'],
            'winsorize': result['winsorize'],
            'features': ', '.join(result['features']) if result['features'] else 'all',
            'RMSE': result['metrics']['RMSE'],
            'MAPE': result['metrics']['MAPE'],
            'Overfit_Score': result['metrics'].get('Overfit Score', np.nan),
            'method': result['method']
        })
    
    return pd.DataFrame(data)

In [10]:
def calculate_metrics(y_true, y_pred, y_train=None, y_train_pred=None, print_metrics=False):
    metrics = {
        'RMSE': np.sqrt(mean_squared_error(y_true, y_pred)),
        'MAPE': mean_absolute_percentage_error(y_true, y_pred) * 100
    }
    
    if y_train is not None and y_train_pred is not None:
        train_rmse = np.sqrt(mean_squared_error(y_train, y_train_pred))
        metrics['Overfit Score'] = (metrics['RMSE'] - train_rmse) / max(train_rmse, 1e-10)
    
    if print_metrics:
        print("\n=== Metrics ===")
        print(f"RMSE: {metrics['RMSE']:.3f}")
        print(f"MAPE: {metrics['MAPE']:.2f}%")
        if 'Overfit Score' in metrics:
            print(f"Overfit Score: {metrics['Overfit Score']:.3f}")
    
    return metrics

def convert_results_to_df(results):
    """Convert results to DataFrame format"""
    if results is None:
        return pd.DataFrame()
    
    return pd.DataFrame([{
        'product_id': results['product_id'],
        'winsorize': results['winsorize'],
        'features': ', '.join(results['features']) if results['features'] else 'all',
        'RMSE': results['metrics']['RMSE'],
        'MAPE': results['metrics']['MAPE'],
        'Overfit_Score': results['metrics'].get('Overfit Score', np.nan),
        'method': results['method']
    }])

#### <span style="background-color:#000027; padding:5px; border-radius:5px;">**3. Model Selection per Product**</span> <a id='model-selection'></a>  

Click [here](#table-of-contents) ⬆️ to return to the Index.


➡️ Our pipeline is going to use:

- **Walk-Forward Validation:** One-step forecasting where the model predicts the next step, and the validation window moves forward one step at a time.
- In the cases we needed to use **GridSearch**, we use a PredefinedSplit method to ensure no cross validation and prevent messing up with the train/validation timeline.

➡️ For Feature Combinations we will iterate over:

- No macro/lags features (baseline model).
- Single-feature models.
- Multi-feature models.

`IMPORTANT:` For feature selection, we must be extremely careful because our dataset is limited—we only have 43 monthly data points per product. This small sample size restricts our modeling options. For initial validation, we’ll use a 7-month forecast horizon because TimeGPT requires at least 36 data points to perform well in a walk-forward validation setup. To ensure fair comparisons across all models, we’ll maintain this 7-month horizon, even though our final goal is to predict 10 months ahead. Given the scarcity of data, we must also limit the number of features to avoid overfitting. Including too many variables could cause the model to capture noise rather than true patterns, reducing its accuracy on unseen data.

➡️ Models

We will tune hyperparameters for all the models for each product, with the best features.


**Time Series Model Comparison Table**

| Feature               | TimeGPT (Zero-Shot) | XGBoost            | TCN                | ARIMA              | Prophet (Facebook)     |
|-----------------------|---------------------|--------------------|--------------------|--------------------|--------------------|
| **Scaling Needed?**   | ❌ No               | ❌ No              | ✅ Yes             | ❌ No (if stationary) | ❌ No (logistic/linear trends handle scaling internally)            |
| **Handles Trends**    | ✅ Excellent        | ✅ (With features) | ✅ Excellent       | ✅ (With differencing) | ✅ Piecewise linear/logistic     |
| **Handles Seasonality** | ✅ Automatic       | ❌ (Requires manual lags) | ✅ (With large receptive field) | ✅ (SARIMA) | ✅ Fourier terms (fixed but interpretable)      |
| **Multivariate Support** | ✅ Yes            | ✅ Yes             | ✅ Yes             | ❌ Univariate only | Limited            |
| **Training Speed**    | ⚡ Instant (pre-trained) | 🏎️ Fast          | 🐢 Slow (GPU helps) | 🏃 Moderate        |  🏎️ Fast          |
| **Interpretability**  | ❌ Black box        | ✅ Feature importances | ❌ Black box     | ✅ Model coefficients | ✅ Best-in-class (clear trend/seasonality plots) |
| **Best For**          | Zero-shot forecasting | Tabular data with temporal features | Long-range dependencies | Simple univariate series | Business-friendly forecasts with explainability |
| **Weakness**          | Limited control     | Poor with long sequences | Computationally heavy | Linear assumptions only | Struggles with short-term dependencies (no AR terms)     |


1. **TimeGPT** (by Nixtla):
   - Pros: No training needed, handles any frequency, great for quick benchmarks.
   - Cons: Proprietary, limited customization.

2. **XGBoost**:
   - Best when you have:
     - Easy and fast. 

3. **TCN** (Temporal Convolutional Networks):
   - Pros: Captures long-range patterns better than LSTMs.
   - Cons: Needs careful tuning of and is more computationaly expensive.

4. **ARIMA**:
   - Important:
     - Data is stationary (or differenced) 
     - Short-term forecasts (≤12 steps)
   - Warning: Fails miserably with:
     - Non-linear trends
     - High-frequency data, wich is ok, bc we are dealing with months.

5. **Prophet**:
   - Faster and more interpretable than NeuralProphet.
   - No auto-regression (unlike NeuralProphet’s AR-Net).
   -Less flexible seasonality (fixed Fourier terms vs. NN-backed).


➡️ Summary:

- **We will Scale**: TCN, Neural Prophet
- **Optional Scaling**: XGBoost (for extreme values)
- **We will not Scale**: ARIMA (but difference if non-stationary!), TimeGPT

1. We will start with **TimeGPT** for a baseline.
2. Then test **XGBoost** with lag features.
3. Then **TCN** to see he effect for more complex patterns.
4. **Prophet** and **ARIMA** for interpretability.

➡️ Evaluation Metrics: 
- RMSE → Measures error magnitude.
- MAPE → Percentage-based error, useful for business impact.
- R² → Measures how well the model explains variance.
- Overfit Score →  

$$
\text{Overfit Score} = \frac{\text{Test RMSE} - \text{Train RMSE}}{\text{Train RMSE}}
$$



    - < 0.1 (Underfit)
    - 0.1 - 0.5 (Good Fit)
    - > 0.5 (Overfit Warning!)

##### <span style="background-color:#000027; padding:5px; border-radius:5px;">**3.1. TimeGPT**</span> <a id='timegpt'></a>  

Click [here](#table-of-contents) ⬆️ to return to the Index.


In [26]:
nixtla_client = NixtlaClient(
    api_key = "nixak-CIwSKQ0cRLIuFR1TYllLFVakTGx3WCY30YPEKfxG0lDQcE0akGo3GE4aMJO9XXbkKjdFaGDP5x6uSxQ6"
)
nixtla_client.validate_api_key()

INFO:nixtla.nixtla_client:Happy Forecasting! :)


True

In [27]:
def choose_parameters_timegpt(product_id, df_train, feature_set, target_col="Sales", 
                              nixtla_client=None, winsorize=False, 
                              horizon=7):
    
    # --- Input Validation ---
    if nixtla_client is None:
        raise ValueError("NixtlaClient instance is required.")
    if len(df_train) < 12:
        print(f"⚠️ Skipping {product_id} - only {len(df_train)} observations")
        return None
    
    # --- Data Preparation ---
    print('Preparing data...')
    try:
        X_train, X_val, y_train, y_val = prepare_time_series_data(df_train, feature_set, target_col, horizon, winsorize)

    except Exception as e:
        print(f"Data preparation failed: {str(e)}")
        return None
 
    # --- Determine Feature Types ---
    hist_exog = [f for f in feature_set if '_lag_' in f or '_ma_' in f] if feature_set else []
    fut_exog = [f for f in feature_set if f not in hist_exog] if feature_set else []


    # --- Prepare Future Exogenous Features ---
    X_future = None
    if fut_exog:
        X_future = X_val[fut_exog].copy()
        if not X_future.isnull().all().all():  
            X_future.insert(0, 'month_year', X_val.index)
        else:
            X_future = None  # Avoid passing empty DataFrame

    # --- Simple Forecast (12 ≤ obs < 36) ---
    if len(X_train) < 36:
        print(f"Using simple forecast ({len(X_train)} < 36 obs)")
        try:
            history = df_train.copy()
            if feature_set:
                history = history[feature_set + [target_col]]

            forecast = nixtla_client.forecast(df=history.reset_index(),
                                              time_col="month_year", target_col=target_col, h=horizon,
                                              X_df=X_future,  # Always pass X_future if available
                                              hist_exog_list=hist_exog if hist_exog else None)
            
            # Get training predictions
            train_pred = nixtla_client.forecast(df=history.reset_index(),time_col="month_year",
                                                target_col=target_col,h=1,hist_exog_list=hist_exog if hist_exog else None
                                                ).iloc[0]["TimeGPT"]
            
            train_predictions = [train_pred] * len(history)
            
            return {
                'product_id': product_id,'winsorize': winsorize,'features': feature_set,
                'metrics': calculate_metrics(y_true=y_val,y_pred=forecast["TimeGPT"],
                                             y_train=history[target_col],y_train_pred=train_predictions),
                'method': 'simple','validation_predictions': forecast["TimeGPT"]}
        except Exception as e:
            print(f"Simple forecast failed: {str(e)}")
            return None


    # --- Walk-Forward (≥36 obs) ---
    print(f"Using walk-forward ({len(X_train)} ≥ 36 obs)")
    predictions = []
    history = X_train.copy()
    history[target_col] = y_train.values
    history = history.reset_index()
    
    # Get training predictions
    try:
        train_fit = nixtla_client.forecast(df=history,time_col='month_year',target_col=target_col,h=1,
                                           hist_exog_list=hist_exog if hist_exog else None)
        train_predictions = [train_fit["TimeGPT"].iloc[0]] * len(y_train)
    except Exception as e:
        print(f"Training pred failed: {str(e)} - using last value")
        train_predictions = [y_train.iloc[-1]] * len(y_train)

    for i in range(horizon):
        try:
            X_future_step = None
            if fut_exog:
                X_future_step = X_val.iloc[[i]][fut_exog].copy()
                if not X_future_step.isnull().all().all():  
                    X_future_step.insert(0, 'month_year', X_val.index[i])
                else:
                    X_future_step = None  

            forecast = nixtla_client.forecast(df=history,time_col='month_year',target_col=target_col,
                                              h=1, X_df=X_future_step if fut_exog else None, # FUTURE features (macro)
                                              hist_exog_list=hist_exog if hist_exog else None)  # HISTORICAL features (lags, moving averages)
            
            pred = forecast["TimeGPT"].iloc[0]
            predictions.append(pred)
            
            new_row = X_val.iloc[i].copy()
            new_row['month_year'] = X_val.index[i]
            new_row[target_col] = pred
            history = pd.concat([history, pd.DataFrame([new_row])])
        
        except Exception as e:
            print(f"Iteration {i+1} failed: {str(e)}")
            raise
    
    metrics = calculate_metrics(y_true=y_val,y_pred=predictions,y_train=y_train,y_train_pred=train_predictions)
    
    return {'product_id': product_id,'winsorize': winsorize,'features': feature_set,'metrics': metrics,
            'method': 'walkforward','validation_predictions': predictions}


def find_best_timegpt_config(product_id, df_train, nixtla_client, lagged_df=None, target_col="Sales"):

    # Validate input data
    if not isinstance(df_train.index, pd.DatetimeIndex):
        raise TypeError("❌ df_train index is not a DatetimeIndex!")
    if df_train.index.isnull().any():
        raise ValueError("❌ Some index values could not be parsed into datetime!")
    
    results = []
    data = df_train.copy()
    
    # Feature groups for original data
    normal_features = [col for col in data.columns if col != target_col]
    normal_feature_combinations = generate_feature_combinations(normal_features, max_features=6)  
    normal_feature_combinations = [[]] + normal_feature_combinations
    print(normal_feature_combinations)
    
    # Test original data configurations
    for winsorize in [True, False]:
        for features in normal_feature_combinations:
            try:
                print(f"\nTesting - Winsorize: {winsorize}, Features: {features}")

                result = choose_parameters_timegpt(
                    product_id=product_id,df_train=data, feature_set=features, target_col=target_col,
                    nixtla_client=nixtla_client,winsorize=winsorize)
                
                if result:
                    print(f"✅ Success - RMSE: {result['metrics']['RMSE']:.2f}")
                    results.append(result)
            except Exception as e:
                print(f"⚠️ Configuration failed: {str(e)}")
                continue
    
    # Test lagged data configurations if available
    if lagged_df is not None:
        lagged_data = lagged_df.copy()
        lag_features = [col for col in lagged_data.columns if 'lag' in col]
        ma_features = [col for col in lagged_data.columns if '_ma_' in col]
        macro_features = [col for col in data.columns if col != target_col]
        
        # Generate lag feature combinations
        lag_feature_combinations = generate_feature_combinations(lag_features, max_features=5)
        
        # Create combined feature sets
        full_combinations = []
        for lag_combo in lag_feature_combinations:
            full_combinations.append(lag_combo)  # Just lags
            full_combinations.append(lag_combo + macro_features)  # Lags + macros
            full_combinations.append(lag_combo + ma_features)  # Lags + MAs
            full_combinations.append(lag_combo + macro_features + ma_features)  # All
        
        # Remove duplicates
        seen = set()
        unique_combinations = []
        for combo in full_combinations:
            combo_tuple = tuple(sorted(combo))
            if combo_tuple not in seen:
                seen.add(combo_tuple)
                unique_combinations.append(combo)
        
        # Test all unique combinations
        for winsorize in [True, False]:
            for features in unique_combinations:
                try:
                    print(f"\nTesting - Lagged Features: {features}")

                    result = choose_parameters_timegpt(product_id=product_id,df_train=lagged_data,feature_set=features,
                                                       target_col=target_col,nixtla_client=nixtla_client,winsorize=winsorize)
                    
                    if result:
                        print(f"✅ Success - RMSE: {result['metrics']['RMSE']:.2f}")
                        results.append(result)
                except Exception as e:
                    print(f"⚠️ Lagged configuration failed: {str(e)}")
                    continue
    
    if not results:
        raise ValueError(f"No valid configurations for {product_id}")
    
    # Select best configuration by RMSE
    best_config = min(results, key=lambda x: x['metrics']['RMSE'])
    
    print(f"\n✅ Best configuration for {product_id}:")
    print(f"- Winsorize: {best_config['winsorize']}")
    print(f"- Features: {best_config['features']}")
    print(f"- RMSE: {best_config['metrics']['RMSE']:.2f}")
    print(f"- Method: {best_config.get('method', 'walkforward')}")
    
    return best_config, results

def process_product_parallel(product_id):
    try:
        df_train = product_dfs[product_id].rename(columns={product_id: "Sales"})
        
        # Ensure DatetimeIndex
        if not isinstance(df_train.index, pd.DatetimeIndex):
            df_train.index = pd.to_datetime(df_train.index, errors="coerce")
        
        if df_train.index.isnull().any():
            raise ValueError("❌ Some index values could not be parsed into datetime!")

        lagged_df = lagged_product_dfs.get(product_id, None)

        best_config, results = find_best_timegpt_config(product_id=product_id,df_train=df_train,lagged_df=lagged_df,
            nixtla_client=nixtla_client,target_col="Sales")
        
        return best_config, results
    except Exception as e:
        print(f"⚠️ Failed to process {product_id}: {str(e)}")
        return None

##### <span style="background-color:#000027; padding:5px; border-radius:5px;">**3.2. XGBoost**</span> <a id='xgboost'></a>  


Click [here](#table-of-contents) ⬆️ to return to the Index.


In [14]:
def choose_parameters_xgboost(product_id, df_train, feature_set, target_col="Sales", 
                            param_grid=None, n_jobs=-1, winsorize=False, horizon=7):
    """Optimized XGBoost forecasting with proper walk-forward validation"""
    
    # --- Input Validation ---
    if len(df_train) < 12:
        print(f"⚠️ Skipping {product_id} - only {len(df_train)} observations")
        return None
    
    # --- Data Preparation ---
    print('Preparing data...')
    try:
        X_train, X_val, y_train, y_val = prepare_time_series_data(
            df_train, feature_set, target_col, horizon, winsorize)
        # Verify temporal ordering
        assert X_train.index[-1] < X_val.index[0], "Validation data must be after training data"
    except Exception as e:
        print(f"Data preparation failed: {str(e)}")
        return None

    # --- Optimized Parameter Grid for Small Data ---
    if param_grid is None:
        param_grid = {
            'n_estimators': [50, 100],
            'max_depth': [2, 3],
            'learning_rate': [0.01, 0.05, 0.1],
            'subsample': [0.9, 1.0],
            'colsample_bytree': [0.9, 1.0],
            'min_child_weight': [1, 3]}

    # --- Efficient Model Setup ---
    base_model = xgb.XGBRegressor(
        objective='reg:squarederror',random_state=42,n_jobs=1,tree_method='hist',  # Faster training
        enable_categorical=False,gamma=0,reg_alpha=0,reg_lambda=1)

    # --- Strict Temporal Validation ---
    test_fold = np.array([-1] * len(X_train) + [0] * len(X_val))
    ps = PredefinedSplit(test_fold)

    # --- Focused Grid Search ---
    grid_search = GridSearchCV(
        estimator=base_model,param_grid=param_grid,scoring='neg_root_mean_squared_error',cv=ps,n_jobs=n_jobs,verbose=1,refit=True)

    # --- Training with Validation ---
    X_combined = pd.concat([X_train, X_val])
    y_combined = pd.concat([y_train, y_val])
    
    print(f"Starting grid search for {product_id}...")
    try:
        grid_search.fit(X_combined, y_combined)
        best_model = grid_search.best_estimator_
    except Exception as e:
        print(f"Grid search failed: {str(e)}")
        return None

    # --- Robust Walk-forward Validation ---
    predictions = []
    print(f"Using walk-forward (n_train={len(X_train)}, horizon={horizon})")
    
    for i in range(horizon):
        try:
            # Create expanding window
            window_X = pd.concat([X_train, X_val.iloc[:i]])
            window_y = pd.concat([y_train, y_val.iloc[:i]])
            
            # Correct XGBoost 2.1.3+ fitting
            best_model.fit(window_X, window_y, eval_set=[(X_val.iloc[[i]], y_val.iloc[[i]])], verbose=0)
    
            pred = best_model.predict(X_val.iloc[[i]]).item()
            predictions.append(pred)
            
        except Exception as e:
            print(f"Iteration {i+1} failed: {str(e)} - using fallback")
            fallback = (predictions[-1] if len(predictions) > 0 else 
                       y_train.iloc[-1] if len(y_train) > 0 else np.nan)
            predictions.append(fallback)

    # --- Training Metrics ---
    try:
        train_predictions = best_model.predict(X_train).tolist()
    except Exception as e:
        print(f"Training predictions failed: {str(e)} - using last value")
        train_predictions = [y_train.iloc[-1]] * len(y_train)

    # --- Comprehensive Metrics ---
    metrics = calculate_metrics(y_true=y_val,y_pred=predictions,y_train=y_train,y_train_pred=train_predictions)
    
    return {
        'product_id': product_id,
        'winsorize': winsorize,
        'features': feature_set,
        'metrics': metrics,
        'method': 'walkforward',
        'best_params': grid_search.best_params_,
        'validation_predictions': predictions,
        'feature_importances': dict(zip(feature_set, best_model.feature_importances_))}


def find_best_xgboost_config(product_id, df_train, lagged_df=None, target_col="Sales", 
                            n_jobs=-1, custom_param_grids=None):
    # Validate input data
    if not isinstance(df_train.index, pd.DatetimeIndex):
        raise TypeError("❌ df_train index is not a DatetimeIndex!")
    if df_train.index.isnull().any():
        raise ValueError("❌ Some index values could not be parsed into datetime!")
    
    results = []
    data = df_train.copy()
    
    # Feature groups for original data
    normal_features = [col for col in data.columns if col != target_col]
    normal_feature_combinations = generate_feature_combinations(normal_features, max_features=6)  
    normal_feature_combinations = [[]] + normal_feature_combinations
    
    # Test original data configurations
    for winsorize in [True, False]:
        for features in normal_feature_combinations:
            try:
                print(f"\nTesting - Winsorize: {winsorize}, Features: {features}")

                # Use custom param grid if provided for this feature set
                param_grid = None
                if custom_param_grids and tuple(features) in custom_param_grids:
                    param_grid = custom_param_grids[tuple(features)]

                result = choose_parameters_xgboost(product_id=product_id,df_train=data,feature_set=features,
                    target_col=target_col,winsorize=winsorize,n_jobs=n_jobs,param_grid=param_grid)
                
                if result:
                    print(f"✅ Success - RMSE: {result['metrics']['RMSE']:.2f}")
                    results.append(result)
            except Exception as e:
                print(f"⚠️ Configuration failed: {str(e)}")
                continue
    
    # Test lagged data configurations if available
    if lagged_df is not None:
        lagged_data = lagged_df.copy()
        lag_features = [col for col in lagged_data.columns if 'lag' in col]
        ma_features = [col for col in lagged_data.columns if '_ma_' in col]
        macro_features = [col for col in data.columns if col != target_col]
        
        # Generate lag feature combinations
        lag_feature_combinations = generate_feature_combinations(lag_features, max_features=5)
        
        # Create combined feature sets
        full_combinations = []
        for lag_combo in lag_feature_combinations:
            full_combinations.append(lag_combo)  # Just lags
            full_combinations.append(lag_combo + macro_features)  # Lags + macros
            full_combinations.append(lag_combo + ma_features)  # Lags + MAs
            full_combinations.append(lag_combo + macro_features + ma_features)  # All
        
        # Remove duplicates
        seen = set()
        unique_combinations = []
        for combo in full_combinations:
            combo_tuple = tuple(sorted(combo))
            if combo_tuple not in seen:
                seen.add(combo_tuple)
                unique_combinations.append(combo)
        
        # Test all unique combinations
        for winsorize in [True, False]:
            for features in unique_combinations:
                try:
                    print(f"\nTesting - Lagged Features: {features}")

                    # Use custom param grid if provided for this feature set
                    param_grid = None
                    if custom_param_grids and tuple(features) in custom_param_grids:
                        param_grid = custom_param_grids[tuple(features)]

                    result = choose_parameters_xgboost(product_id=product_id,df_train=lagged_data,feature_set=features,
                                                       target_col=target_col,winsorize=winsorize,n_jobs=n_jobs,param_grid=param_grid)
                    
                    if result:
                        print(f"✅ Success - RMSE: {result['metrics']['RMSE']:.2f}")
                        results.append(result)
                except Exception as e:
                    print(f"⚠️ Lagged configuration failed: {str(e)}")
                    continue
    
    if not results:
        raise ValueError(f"No valid configurations for {product_id}")
    
    # Select best configuration by RMSE
    best_config = min(results, key=lambda x: x['metrics']['RMSE'])
    
    print(f"\n✅ Best configuration for {product_id}:")
    print(f"- Winsorize: {best_config['winsorize']}")
    print(f"- Features: {best_config['features']}")
    print(f"- Best params: {best_config['best_params']}")
    print(f"- RMSE: {best_config['metrics']['RMSE']:.2f}")
    print(f"- Method: {best_config.get('method', 'walkforward')}")
    
    return best_config, results

def process_product_parallel(product_id, custom_param_grids=None):
    try:
        df_train = product_dfs[product_id].rename(columns={product_id: "Sales"})
        
        # Ensure DatetimeIndex
        if not isinstance(df_train.index, pd.DatetimeIndex):
            df_train.index = pd.to_datetime(df_train.index, errors="coerce")
        
        if df_train.index.isnull().any():
            raise ValueError("❌ Some index values could not be parsed into datetime!")

        lagged_df = lagged_product_dfs.get(product_id, None)

        best_config, results = find_best_xgboost_config(product_id=product_id,df_train=df_train,lagged_df=lagged_df,target_col="Sales", custom_param_grids=custom_param_grids)
        
        return best_config, results
    except Exception as e:
        print(f"⚠️ Failed to process {product_id}: {str(e)}")
        return None

##### <span style="background-color:#000027; padding:5px; border-radius:5px;">**3.3. TCN**</span> <a id='tcn'></a>  

Click [here](#table-of-contents) ⬆️ to return to the Index.


In [15]:
def choose_parameters_tcn(product_id, df_train, feature_set, target_col="Sales", 
                         param_grid=None, n_jobs=-1, winsorize=False, horizon=7):
    """Optimized TCN forecasting with walk-forward validation"""
    
    if len(df_train) < 12:
        print(f"⚠️ Skipping {product_id} - only {len(df_train)} observations")
        return None
    
    print('Preparing data...')
    try:
        X_train, X_val, y_train, y_val = prepare_time_series_data(
            df_train, feature_set, target_col, horizon, winsorize, scaling=True)
        
        assert X_train.index[-1] < X_val.index[0], "Validation data must be after training data"
    except Exception as e:
        print(f"Data preparation failed: {str(e)}")
        return None

    # Reshape data for TCN input (samples, timesteps, features)
    X_train = X_train.values.reshape((X_train.shape[0], 1, X_train.shape[1]))
    X_val = X_val.values.reshape((X_val.shape[0], 1, X_val.shape[1]))
    y_train = y_train.values.reshape(-1, 1)
    y_val = y_val.values.reshape(-1, 1)

    if param_grid is None:
        param_grid = {
            'nb_filters': [16, 32],
            'kernel_size': [2, 3],
            'dilations': [[1, 2, 4], [1, 2, 4, 8]],
            'dropout_rate': [0.1, 0.2],
            'return_sequences': [False]  # single-step prediction
}
    
    def build_tcn(nb_filters=32, kernel_size=2, dilations=[1, 2, 4], 
                 dropout_rate=0.1, return_sequences=False):
        inputs = Input(shape=(1, X_train.shape[2]))
        x = TCN(nb_filters=nb_filters, kernel_size=kernel_size, 
               dilations=dilations, dropout_rate=dropout_rate,
               return_sequences=return_sequences)(inputs)
        outputs = Dense(1)(x)
        model = Model(inputs, outputs)
        model.compile(optimizer="adam", loss="mse")
        return model
    
    best_params = None
    best_rmse = float('inf')

    for filters in param_grid['nb_filters']:
        for kernel in param_grid['kernel_size']:
            for dilations in param_grid['dilations']:
                for dropout in param_grid['dropout_rate']:
                    for return_seq in param_grid['return_sequences']:
                        model = build_tcn(filters, kernel, dilations, dropout, return_seq)
                        try:
                            model.fit(X_train, y_train, epochs=50, batch_size=16, verbose=0)
                            
                            # Walk-forward validation (predict one step at a time)
                            predictions = []
                            history_X = X_train.copy()
                            history_y = y_train.copy()
                            
                            for i in range(len(X_val)):
                                # Predict next step
                                X_next = X_val[i].reshape(1, 1, -1)
                                pred = model.predict(X_next, verbose=0).flatten()[0]
                                predictions.append(pred)
                                
                            # Calculate RMSE
                            if len(y_val) == len(predictions):
                                rmse = np.sqrt(np.mean((y_val.flatten() - predictions) ** 2))
                            else:
                                print(f"Shape mismatch: y_val ({y_val.shape}), preds ({len(predictions)})")
                                continue
                            
                            if rmse < best_rmse:
                                best_rmse = rmse
                                best_params = {
                                    'nb_filters': filters,
                                    'kernel_size': kernel,
                                    'dilations': dilations,
                                    'dropout_rate': dropout,
                                    'return_sequences': return_seq
                                }
                        except Exception as e:
                            print(f"Hyperparameter set failed: {str(e)}")

    if best_params is None:
        print(f"❌ No valid models for {product_id}")
        return None

    # Final model training and walk-forward prediction
    final_model = build_tcn(**best_params)
    final_model.fit(X_train, y_train, epochs=50, batch_size=16, verbose=0)
    predictions = []

    # FIXED: Create DataFrame with both features and target
    history_data = np.column_stack([X_train[:, 0, :], y_train.flatten()])
    history = pd.DataFrame(history_data, columns=feature_set + [target_col])
    history.index = range(len(history))
    
    for i in range(horizon):
        try:
            X_future_step = X_val[i].reshape(1, 1, -1)
            pred = final_model.predict(X_future_step, verbose=0).flatten()[0]
            predictions.append(pred)
            
            # Update training data
            new_row = X_val[i, 0, :].tolist()
            new_row.append(pred)
            new_df = pd.DataFrame([new_row], columns=feature_set + [target_col])
            history = pd.concat([history, new_df]).reset_index(drop=True)
            
            # Retrain on expanded window
            rolling_window = 36 if len(history) > 36 else len(history)
            train_subset = history.iloc[-rolling_window:]
            
            X_train_update = train_subset[feature_set].values.reshape(-1, 1, len(feature_set))
            y_train_update = train_subset[target_col].values.reshape(-1, 1)
            
            final_model.fit(X_train_update, y_train_update, epochs=5, batch_size=16, verbose=0)
        
        except Exception as e:
            print(f"Iteration {i+1} failed: {str(e)}")
            predictions.append(predictions[-1] if predictions else y_train[-1][0])
    
    metrics = calculate_metrics(y_true=y_val.flatten(), y_pred=predictions, y_train=y_train.flatten())
    
    return {
        'product_id': product_id,
        'winsorize': winsorize,
        'features': feature_set,
        'metrics': metrics,
        'method': 'walkforward',
        'best_params': best_params,
        'validation_predictions': predictions
    }

def find_best_tcn_config(product_id, df_train, lagged_df=None, target_col="Sales", 
                            n_jobs=-1, custom_param_grids=None):
    # Validate input data
    if not isinstance(df_train.index, pd.DatetimeIndex):
        raise TypeError("❌ df_train index is not a DatetimeIndex!")
    if df_train.index.isnull().any():
        raise ValueError("❌ Some index values could not be parsed into datetime!")
    
    results = []
    data = df_train.copy()
    
    # Feature groups for original data
    normal_features = [col for col in data.columns if col != target_col]
    normal_feature_combinations = generate_feature_combinations(normal_features, max_features=6)  
    normal_feature_combinations = normal_feature_combinations
    
    # Test original data configurations
    for winsorize in [True, False]:
        for features in normal_feature_combinations:
            try:
                print(f"\nTesting - Winsorize: {winsorize}, Features: {features}")

                # Use custom param grid if provided for this feature set
                param_grid = None
                if custom_param_grids and tuple(features) in custom_param_grids:
                    param_grid = custom_param_grids[tuple(features)]

                result = choose_parameters_tcn(product_id=product_id,df_train=data,feature_set=features,
                    target_col=target_col,winsorize=winsorize,n_jobs=n_jobs,param_grid=param_grid)
                
                if result:
                    print(f"✅ Success - RMSE: {result['metrics']['RMSE']:.2f}")
                    results.append(result)
            except Exception as e:
                print(f"⚠️ Configuration failed: {str(e)}")
                continue
    
    # Test lagged data configurations if available
    if lagged_df is not None:
        lagged_data = lagged_df.copy()
        lag_features = [col for col in lagged_data.columns if 'lag' in col]
        ma_features = [col for col in lagged_data.columns if '_ma_' in col]
        macro_features = [col for col in data.columns if col != target_col]
        
        # Generate lag feature combinations
        lag_feature_combinations = generate_feature_combinations(lag_features, max_features=5)
        
        # Create combined feature sets
        full_combinations = []
        for lag_combo in lag_feature_combinations:
            full_combinations.append(lag_combo)  # Just lags
            full_combinations.append(lag_combo + macro_features)  # Lags + macros
            full_combinations.append(lag_combo + ma_features)  # Lags + MAs
            full_combinations.append(lag_combo + macro_features + ma_features)  # All
        
        # Remove duplicates
        seen = set()
        unique_combinations = []
        for combo in full_combinations:
            combo_tuple = tuple(sorted(combo))
            if combo_tuple not in seen:
                seen.add(combo_tuple)
                unique_combinations.append(combo)
        
        # Test all unique combinations
        for winsorize in [True, False]:
            for features in unique_combinations:
                try:
                    print(f"\nTesting - Lagged Features: {features}")

                    # Use custom param grid if provided for this feature set
                    param_grid = None
                    if custom_param_grids and tuple(features) in custom_param_grids:
                        param_grid = custom_param_grids[tuple(features)]

                    result = choose_parameters_tcn(product_id=product_id,df_train=lagged_data,feature_set=features,
                                                       target_col=target_col,winsorize=winsorize,n_jobs=n_jobs,param_grid=param_grid)
                    
                    if result:
                        print(f"✅ Success - RMSE: {result['metrics']['RMSE']:.2f}")
                except Exception as e:
                    print(f"⚠️ Lagged configuration failed: {str(e)}")
                    continue
    
    if not results:
        raise ValueError(f"No valid configurations for {product_id}")
    
    # Select best configuration by RMSE
    best_config = min(results, key=lambda x: x['metrics']['RMSE'])
    
    print(f"\n✅ Best configuration for {product_id}:")
    print(f"- Winsorize: {best_config['winsorize']}")
    print(f"- Features: {best_config['features']}")
    print(f"- Best params: {best_config['best_params']}")
    print(f"- RMSE: {best_config['metrics']['RMSE']:.2f}")
    print(f"- Method: {best_config.get('method', 'walkforward')}")
    
    return best_config, results

def process_product_parallel(product_id, custom_param_grids=None):
    try:
        df_train = product_dfs[product_id].rename(columns={product_id: "Sales"})
        
        # Ensure DatetimeIndex
        if not isinstance(df_train.index, pd.DatetimeIndex):
            df_train.index = pd.to_datetime(df_train.index, errors="coerce")
        
        if df_train.index.isnull().any():
            raise ValueError("❌ Some index values could not be parsed into datetime!")

        lagged_df = lagged_product_dfs.get(product_id, None)
        best_config, results = find_best_tcn_config(product_id=product_id,df_train=df_train,lagged_df=lagged_df,target_col="Sales", custom_param_grids=custom_param_grids)
        
        return best_config, results
    except Exception as e:
        print(f"⚠️ Failed to process {product_id}: {str(e)}")
        return None

In [16]:
all_product_ids = set(product_dfs.keys())
all_results_tcn = []  # List to store all results
best_configs_tcn = []  # List to store best configurations

for product_id in all_product_ids:
    print(f"Processing product: {product_id}")
    
    try:
        best_config_tcn, results_tcn = process_product_parallel(product_id, custom_param_grids=None)
        
        if best_config_tcn is not None:
            best_configs_tcn.append(best_config_tcn)  # Store best config
        else:
            print(f"Warning: No best config returned for product {product_id}")
        
        if results_tcn is not None:
            all_results_tcn.extend(results_tcn)  # Store results
        else:
            print(f"Warning: No results returned for product {product_id}")
            
    except Exception as e:
        print(f"Error processing product {product_id}: {str(e)}")

Processing product: P1

Testing - Winsorize: True, Features: ['MAB_ELE_SHP840']
Preparing data...
Sales: Winsorized Bounds -> Lower = 12773011.92, Upper = 41955264.27


KeyboardInterrupt: 

##### <span style="background-color:#000027; padding:5px; border-radius:5px;">**3.4. Prophet**</span> <a id='prophet'></a>  

Click [here](#table-of-contents) ⬆️ to return to the Index.

In [56]:
def get_product_specific_config(product_id):
    """Returns Prophet configuration tailored to each product's time series characteristics"""
    product_configs = {
        'P1': {
            'seasonality_mode': 'additive',
            'yearly_seasonality': False,
            'weekly_seasonality': False,
            'daily_seasonality': False,
            'changepoint_prior_scale': 0.05,  
            'seasonality_prior_scale': 10.0,
            'extra_regressors': []},
        'P3': {
            'seasonality_mode': 'multiplicative',
            'yearly_seasonality': True,
            'weekly_seasonality': False,
            'daily_seasonality': False,
            'changepoint_prior_scale': 0.1,
            'seasonality_prior_scale': 15.0, 
            'extra_regressors': []},
        'P4': {
            'seasonality_mode': 'additive',
            'yearly_seasonality': False,
            'weekly_seasonality': False,
            'daily_seasonity': False,
            'changepoint_prior_scale': 0.05,
            'seasonality_prior_scale': 10.0,
            'extra_seasonalities': [{
                'name': 'semiannual',
                'period': 6,
                'fourier_order': 3}]},
        'P5': {
            'seasonality_mode': 'multiplicative',
            'yearly_seasonality': False,
            'weekly_seasonality': False,
            'daily_seasonality': False,
            'changepoint_prior_scale': 0.05,
            'seasonality_prior_scale': 10.0,
            'extra_seasonalities': [{
                'name': 'semiannual',
                'period': 6,
                'fourier_order': 3}]},
        'P6': {
            'seasonality_mode': 'additive',
            'yearly_seasonality': False,
            'weekly_seasonality': False,
            'daily_seasonality': False,
            'changepoint_prior_scale': 0.05,  # gradual changes
            'seasonality_prior_scale': 10.0,
            'extra_regressors': []},
        'P8': {
            'seasonality_mode': 'additive',
            'yearly_seasonality': False,
            'weekly_seasonality': False,
            'daily_seasonality': False,
            'changepoint_prior_scale': 0.2,
            'seasonality_prior_scale': 15.0,
            'extra_seasonalities': [{
                'name': 'quarterly',
                'period': 3,
                'fourier_order': 3}]},
        'P9': {
            'seasonality_mode': 'multiplicative',
            'yearly_seasonality': True,
            'weekly_seasonality': False,
            'daily_seasonality': False,
            'changepoint_prior_scale': 0.1,
            'seasonality_prior_scale': 15.0,  # Strong yearly anti-correlation
            'extra_regressors': []},
        'P11': {
            'seasonality_mode': 'multiplicative',
            'yearly_seasonality': True,
            'weekly_seasonality': False,
            'daily_seasonality': False,
            'changepoint_prior_scale': 0.15,
            'seasonality_prior_scale': 20.0,  # Very strong yearly dependency
            'extra_regressors': []},
        'P12': {
            'seasonality_mode': 'additive',
            'yearly_seasonality': False,
            'weekly_seasonality': False,
            'daily_seasonality': False,
            'changepoint_prior_scale': 0.3,  # For strong short-term dependencies
            'seasonality_prior_scale': 5.0,
            'extra_regressors': []},
        'P13': {
            'seasonality_mode': 'additive',
            'yearly_seasonality': False,
            'weekly_seasonality': False,
            'daily_seasonality': False,
            'changepoint_prior_scale': 0.1,
            'seasonality_prior_scale': 15.0,
            'extra_seasonalities': [{
                'name': 'custom_17',
                'period': 17,
                'fourier_order': 3}]},
        'P14': {
            'seasonality_mode': 'additive',
            'yearly_seasonality': False,
            'weekly_seasonality': False,
            'daily_seasonality': False,
            'changepoint_prior_scale': 0.05,
            'seasonality_prior_scale': 10.0,
            'extra_seasonalities': [{
                'name': 'semiannual',
                'period': 6,
                'fourier_order': 3}]},
        'P16': {
            'seasonality_mode': 'additive',
            'yearly_seasonality': False,
            'weekly_seasonality': False,
            'daily_seasonality': False,
            'changepoint_prior_scale': 0.3,  # For strong ARMA(3,3) patterns
            'seasonality_prior_scale': 5.0,
            'extra_regressors': []},
        'P20': {
            'seasonality_mode': 'additive',
            'yearly_seasonality': False,
            'weekly_seasonality': False,
            'daily_seasonality': False,
            'changepoint_prior_scale': 0.1,
            'seasonality_prior_scale': 15.0,
            'extra_seasonalities': [{
                'name': 'custom_13',
                'period': 13,
                'fourier_order': 3}]},
        'P36': {
            'seasonality_mode': 'multiplicative',
            'yearly_seasonality': False,
            'weekly_seasonality': False,
            'daily_seasonality': False,
            'changepoint_prior_scale': 0.1,
            'seasonality_prior_scale': 15.0,
            'extra_seasonalities': [{
                'name': 'custom_10',
                'period': 10,
                'fourier_order': 3}]}}
    
    # Default configuration 
    default_config = {
        'seasonality_mode': 'additive',
        'yearly_seasonality': 'auto',
        'weekly_seasonality': False,
        'daily_seasonality': False,
        'changepoint_prior_scale': 0.05,
        'seasonality_prior_scale': 10.0,
        'extra_regressors': []}
    
    return product_configs.get(product_id, default_config)

In [58]:
def choose_parameters_prophet(product_id, df_train, feature_set, target_col="Sales", 
                             winsorize=False, horizon=7):
    
    # --- Input Validation ---
    if len(df_train) < 12:
        print(f"⚠️ Skipping {product_id} - only {len(df_train)} observations")
        return None
    
    # --- Data Preparation ---
    print('Preparing data...')
    try:
        X_train, X_val, y_train, y_val = prepare_time_series_data(
            df_train, feature_set, target_col, horizon, winsorize
        )

    except Exception as e:
        print(f"Data preparation failed: {str(e)}")
        return None
    
    # Get product-specific configuration
    config = get_product_specific_config(product_id)
    
    # Prepare Prophet-compatible DataFrames
    train_df = pd.DataFrame({
        'ds': X_train.index.to_numpy(),
        'y': y_train.values.flatten()
    })
    
    # Add regressors if any
    regressors = []
    if feature_set:
        for feature in feature_set:
            if feature in X_train.columns:
                train_df[feature] = X_train[feature].values.reshape(-1)
                regressors.append(feature)
    
    # --- Model Configuration Function ---
    def create_prophet_model():
        """Helper to create a fresh Prophet instance with current config"""
        model = Prophet(seasonality_mode=config['seasonality_mode'], yearly_seasonality=config['yearly_seasonality'], weekly_seasonality=config['weekly_seasonality'],
            daily_seasonality=config['daily_seasonality'], changepoint_prior_scale=config['changepoint_prior_scale'], seasonality_prior_scale=config['seasonality_prior_scale'])
        
        # custom seasonalities
        if 'extra_seasonalities' in config:
            for seasonality in config['extra_seasonalities']:
                model.add_seasonality(name=seasonality['name'], period=seasonality['period'], fourier_order=seasonality['fourier_order'])
        
        # Add regressors
        for reg in regressors:
            model.add_regressor(reg)
        
        return model
    
    # --- Simple Forecast (12 ≤ obs < 36) ---
    if len(X_train) < 36:
        print(f"Using simple forecast ({len(X_train)} < 36 obs)")
        try:
            model = create_prophet_model()
            model.fit(train_df)
            
            future = model.make_future_dataframe(periods=horizon, freq='M')
            
            if regressors and len(X_val) >= horizon:
                for reg in regressors:
                    future[reg] = X_val[reg].iloc[:horizon].values.reshape(-1)
            
            forecast = model.predict(future)
            val_predictions = forecast['yhat'].values[-horizon:].flatten()
            train_predictions = forecast['yhat'].values[:-horizon].flatten() if horizon > 0 else forecast['yhat'].values.flatten()
            
            return {
                'product_id': product_id,
                'winsorize': winsorize,
                'features': feature_set,
                'metrics': calculate_metrics(y_true=y_val.values.flatten(), y_pred=val_predictions, y_train=y_train.values.flatten(), y_train_pred=train_predictions),
                'method': 'simple',
                'validation_predictions': val_predictions,
                'model_config': config}
        except Exception as e:
            print(f"Simple forecast failed: {str(e)}")
            return None
    
    # --- Walk-Forward (≥36 obs) ---
    print(f"Using walk-forward ({len(X_train)} ≥ 36 obs)")
    predictions = []
    actuals = []
    history = train_df.copy()
    history = history.convert_dtypes()

    # Get training predictions
    try:
        train_model = create_prophet_model()
        train_model.fit(history)
        future_train = train_model.make_future_dataframe(periods=0, freq='M')
        if regressors:
            for reg in regressors:
                future_train[reg] = history[reg].values.astype(float)  # Explicit float conversion
        train_forecast = train_model.predict(future_train)
        train_predictions = train_forecast['yhat'].values.flatten()
    except Exception as e:
        print(f"Training pred failed: {str(e)} - using last value")
        train_predictions = np.full(len(y_train), y_train.iloc[-1])

    # Walk-forward validation
    for i in range(horizon):
        try:
            # Create fresh model
            step_model = create_prophet_model()
            
            # Ensure numeric types
            history = history.infer_objects()
            
            # Fit model
            step_model.fit(history)
            
            # Prepare one-step forecast
            future_step = step_model.make_future_dataframe(periods=1, include_history=False, freq='M')
            
            # Add regressors with type checking
            if regressors and i < len(X_val):
                for reg in regressors:
                    if not np.issubdtype(X_val[reg].dtype, np.number):
                        raise TypeError(f"Regressor {reg} contains non-numeric values")
                    future_step[reg] = float(X_val[reg].iloc[i])  # Explicit float conversion
            
            # Make prediction
            forecast_step = step_model.predict(future_step)
            pred = float(forecast_step['yhat'].iloc[0])  # Explicit float conversion
            predictions.append(pred)
            actual = float(y_val.iloc[i]) if i < len(y_val) else np.nan
            actuals.append(actual)
            
            # Update history 

            new_row = {'ds': X_val.index[i], 'y': actual}
            if regressors and i < len(X_val):
                for reg in regressors:
                    new_row[reg] = float(X_val[reg].iloc[i])
            
            history = pd.concat([
                history, 
                pd.DataFrame([new_row]).infer_objects()], ignore_index=True).convert_dtypes()
            
        except Exception as e:
            print(f"Iteration {i+1} failed: {str(e)}")
            predictions.append(float(y_train.iloc[-1]))
            actuals.append(float(y_val.iloc[i]) if i < len(y_val) else np.nan)
    
    # Calculate metrics using actual observed values
    metrics = calculate_metrics(
        y_true=np.array(actuals).flatten(),
        y_pred=np.array(predictions).flatten(),
        y_train=y_train.values.flatten(),
        y_train_pred=train_predictions)
    
    return {
        'product_id': product_id, 'winsorize': winsorize, 'features': feature_set, 'metrics': metrics, 'method': 'walkforward', 'validation_predictions': predictions, 'model_config': config}

def find_best_prophet_config(product_id, df_train, lagged_df=None, target_col="Sales"):
    """Find best Prophet configuration with improved error handling"""
    
    # Validate input data
    if not isinstance(df_train.index, pd.DatetimeIndex):
        raise TypeError("❌ df_train index is not a DatetimeIndex!")
    if df_train.index.isnull().any():
        raise ValueError("❌ Some index values could not be parsed into datetime!")
    
    results = []
    data = df_train.copy()
    
    # Feature groups for original data
    normal_features = [col for col in data.columns if col != target_col]
    normal_feature_combinations = generate_feature_combinations(normal_features, max_features=6)  
    
    # Test original data configurations
    for winsorize in [True, False]:
        for features in normal_feature_combinations:
            try:
                print(f"\nTesting - Winsorize: {winsorize}, Features: {features}")

                result = choose_parameters_prophet(product_id=product_id, df_train=data, feature_set=features, target_col=target_col, winsorize=winsorize)
                
                if result:
                    print(f"✅ Success - RMSE: {result['metrics']['RMSE']:.2f}")
                    results.append(result)
            except Exception as e:
                print(f"⚠️ Configuration failed: {str(e)}")
                continue
    
    # Test lagged data configurations if available
    if lagged_df is not None:
        lagged_data = lagged_df.copy()
        lag_features = [col for col in lagged_data.columns if 'lag' in col]
        ma_features = [col for col in lagged_data.columns if '_ma_' in col]
        macro_features = [col for col in data.columns if col != target_col]
        
        # Generate lag feature combinations
        lag_feature_combinations = generate_feature_combinations(lag_features, max_features=5)
        
        # Create combined feature sets
        full_combinations = []
        for lag_combo in lag_feature_combinations:
            full_combinations.append(lag_combo)
            full_combinations.append(lag_combo + macro_features)
            full_combinations.append(lag_combo + ma_features)
            full_combinations.append(lag_combo + macro_features + ma_features)
        
        # Remove duplicates
        seen = set()
        unique_combinations = []
        for combo in full_combinations:
            combo_tuple = tuple(sorted(combo))
            if combo_tuple not in seen:
                seen.add(combo_tuple)
                unique_combinations.append(combo)
        
        # Test all unique combinations
        for winsorize in [True, False]:
            for features in unique_combinations:
                try:
                    print(f"\nTesting - Lagged Features: {features}")

                    result = choose_parameters_prophet(product_id=product_id, df_train=lagged_data, feature_set=features, target_col=target_col, winsorize=winsorize)
                    
                    if result:
                        print(f"✅ Success - RMSE: {result['metrics']['RMSE']:.2f}")
                        results.append(result)
                except Exception as e:
                    print(f"⚠️ Lagged configuration failed: {str(e)}")
                    continue
    
    if not results:
        raise ValueError(f"No valid configurations for {product_id}")
    
    # Select best configuration by RMSE
    best_config = min(results, key=lambda x: x['metrics']['RMSE'])
    
    print(f"\n✅ Best configuration for {product_id}:")
    print(f"- Winsorize: {best_config['winsorize']}")
    print(f"- Features: {best_config['features']}")
    print(f"- RMSE: {best_config['metrics']['RMSE']:.2f}")
    print(f"- Method: {best_config.get('method', 'walkforward')}")
    
    return best_config, results

def process_product_prophet(product_id):
    """Parallel processing wrapper for NeuralProphet"""
    try:
        df_train = product_dfs[product_id].rename(columns={product_id: "Sales"})
        
        # Ensure DatetimeIndex
        if not isinstance(df_train.index, pd.DatetimeIndex):
            df_train.index = pd.to_datetime(df_train.index, errors="coerce")
        
        if df_train.index.isnull().any():
            raise ValueError("❌ Some index values could not be parsed into datetime!")

        lagged_df = lagged_product_dfs.get(product_id, None)
        
        best_config, results = find_best_prophet_config(
            product_id=product_id,
            df_train=df_train,
            lagged_df=lagged_df,
            target_col="Sales")
        
        return best_config, results
    except Exception as e:
        print(f"⚠️ Failed to process {product_id}: {str(e)}")
        return None

##### <span style="background-color:#000027; padding:5px; border-radius:5px;">**3.5. ARIMA**</span> <a id='arima'></a>  

Click [here](#table-of-contents) ⬆️ to return to the Index.

In [21]:
def choose_parameters_arima(product_id, df_train, feature_set, target_col="Sales", 
                          winsorize=False, horizon=7, seasonal_period=None,
                          manual_order=None, manual_seasonal_order=None):
    """
    ARIMA/SARIMAX with:
    - Support for both manual and auto-ARIMA
    - Proper non-stationary handling
    - Fixed dimensionality issues
    """
    # --- Input Validation ---
    if len(df_train) < 12:
        print(f"⚠️ Skipping {product_id} - only {len(df_train)} observations")
        return None
    
    # --- Data Preparation ---
    print('Preparing data...')
    try:
        X_train, X_val, y_train, y_val = prepare_time_series_data(df_train, feature_set, target_col, horizon, winsorize)
        
        # Convert to numpy arrays and ensure correct shapes
        y_train = np.array(y_train).ravel()
        y_val = np.array(y_val).ravel()
        
    except Exception as e:
        print(f"Data preparation failed: {str(e)}")
        return None
    
    # --- Prepare Exogenous Data ---
    exog_train = np.array(X_train[feature_set]) if feature_set else None
    exog_val = np.array(X_val[feature_set]) if feature_set else None
    
    # --- Walk-Forward Validation ---
    print(f"Using walk-forward validation ({len(X_train)} observations)")
    predictions = []
    history = y_train.copy()
    exog_history = exog_train.copy() if exog_train is not None else None
    
    # Get training predictions
    try:
        if manual_order:  # Manual configuration path
            with warnings.catch_warnings():
                warnings.simplefilter("ignore")

                model = SARIMAX(history, exog=exog_history, order=manual_order,
                              seasonal_order=manual_seasonal_order if manual_seasonal_order else (0,0,0,0),
                              enforce_stationarity=False, enforce_invertibility=False).fit(disp=False)
                
                train_predictions = model.get_prediction().predicted_mean

        else:  # Auto-ARIMA path
            with warnings.catch_warnings():
                warnings.simplefilter("ignore")

                train_model = auto_arima(history, X=exog_history, seasonal=seasonal_period is not None,
                                      m=seasonal_period if seasonal_period else 1, 
                                      suppress_warnings=True, error_action='ignore',
                                      test='adf') # Auto-detect differencing needs
                
                train_predictions = train_model.predict_in_sample(X=exog_history)

    except Exception as e:
        print(f"Training pred failed: {str(e)} - using last value")
        train_predictions = [history[-1]] * len(history)
    
    for i in range(horizon):
        try:
            with warnings.catch_warnings():
                warnings.simplefilter("ignore")
                current_exog = exog_val[[i]] if exog_val is not None else None
                
                if manual_order:  # Manual fit
                    model = SARIMAX(history, exog=exog_history, order=manual_order,
                                  seasonal_order=manual_seasonal_order if manual_seasonal_order else (0,0,0,0),
                                  enforce_stationarity=False).fit(disp=False)
                    
                    pred = model.forecast(steps=1, exog=current_exog)[0]

                else:  # Auto-ARIMA
                    model = auto_arima(history, X=exog_history, seasonal=seasonal_period is not None,
                                     m=seasonal_period if seasonal_period else 1, 
                                     suppress_warnings=True, error_action='ignore')
                    
                    pred = model.predict(n_periods=1, X=current_exog)[0]
                
                predictions.append(pred)
                # Update history maintaining 1D array
                history = np.concatenate([history, [pred]])
                if exog_history is not None:
                    exog_history = np.vstack([exog_history, exog_val[i]])
                    
        except Exception as e:
            print(f"Iteration {i+1} failed: {str(e)}")
            predictions.append(history[-1])
            continue
    
    metrics = calculate_metrics(y_true=y_val, y_pred=predictions, y_train=y_train, y_train_pred=train_predictions)
    
    return {
        'product_id': product_id, 'winsorize': winsorize, 'features': feature_set,  
        'metrics': metrics, 'method': 'manual' if manual_order else 'auto',
        'validation_predictions': predictions, 'model_order': manual_order if manual_order else model.order,
        'seasonal_order': manual_seasonal_order if manual_seasonal_order else (model.seasonal_order if hasattr(model, 'seasonal_order') else (0,0,0,0))}

def find_best_arima_config(product_id, df_train, target_col="Sales"):
    """
    Tests both manual (from ACF/PACF) and auto-ARIMA configurations
    """
        
    #  DatetimeIndex
    if not isinstance(df_train.index, pd.DatetimeIndex):
        df_train.index = pd.to_datetime(df_train.index, errors="coerce")

    # Validate input
    if not isinstance(df_train.index, pd.DatetimeIndex):
        raise TypeError("❌ df_train index is not a DatetimeIndex!")
    
    results = []
    data = df_train.copy()
    
    # --- Manual Configurations from ACF/PACF Analysis ---
    manual_configs = {
        'P1': [{'order': (1,0,0)}], # AR(1)
        'P3': [{'order': (1,0,0), 'seasonal_order': (1,0,0,12)}], # SARIMA(1,0,0)(1,0,0,12)
        'P4': [{'order': (0,0,0), 'seasonal_order': (1,0,0,6)}], # SARIMA(0,0,0)(1,0,0,6)
        'P5': [{'order': (0,0,0), 'seasonal_order': (1,0,1,6)}], # SARIMA(0,0,0)(1,0,1,6)
        'P6': [{'order': (1,0,0)}], # AR(1)
        'P8': [  # Non-stationary series
            {'order': (3,1,3)},  # ARIMA(3,1,3) - with differencing
            {'order': (3,0,3)}], # ARIMA(3,0,3) - without differencing for comparison
        'P9': [{'order': (0,0,0), 'seasonal_order': (0,0,1,12)}], # SARIMA(0,0,0)(0,0,1,12)
        'P11': [{'order': (1,0,0), 'seasonal_order': (1,0,0,12)}], # SARIMA(1,0,0)(1,0,0,12)
        'P12': [{'order': (3,0,0)}],  # AR(3)
        'P13': [
            {'order': (0,0,0), 'seasonal_order': (1,0,0,17)},  # SARIMA(0,0,0)(1,0,0,17)
            {'order': (1,0,0)}], # Also test standard AR(1) as fallback
        'P14': [{'order': (0,0,0), 'seasonal_order': (1,0,0,6)}], # SARIMA(0,0,0)(1,0,0,6)
        'P16': [{'order': (3,0,3)}],  # ARMA(3,3)
        'P20': [{'order': (0,0,0), 'seasonal_order': (1,0,0,13)}], # SARIMA(0,0,0)(1,0,0,13)
        'P36': [{'order': (0,0,0), 'seasonal_order': (1,0,1,10)}]} # SARIMA(0,0,0)(1,0,1,10)
    
    # --- Test Manual Configurations ---
    if product_id in manual_configs:
        for config in manual_configs[product_id]:
            for winsorize in [True, False]:
                try:
                    result = choose_parameters_arima(product_id=product_id, df_train=data,
                                                     feature_set=[f for f in data.columns if f != target_col],
                                                     target_col=target_col,
                                                     winsorize=winsorize,
                                                     manual_order=config['order'],
                                                     manual_seasonal_order=config.get('seasonal_order'))
                    if result:
                        print(f"✅ Manual Success - Order: {result['model_order']} "
                              f"Seasonal: {result['seasonal_order']} "
                              f"RMSE: {result['metrics']['RMSE']:.2f}")
                        results.append(result)
                except Exception as e:
                    print(f"⚠️ Manual config failed: {str(e)}")
    
    # --- Test Auto-ARIMA Configurations ---
    seasonal_periods = {
        'P3': 12, 'P4': 6, 'P5': 6, 'P9': 12,
        'P11': 12, 'P13': 17, 'P14': 6, 'P20': 13, 'P36': 10}
    
    seasonal_period = seasonal_periods.get(product_id)
    
    normal_features = [col for col in data.columns if col != target_col]
    feature_combinations = generate_feature_combinations(normal_features, max_features=6)
    feature_combinations = [[]] + feature_combinations  # Include no-features option
    
    for winsorize in [True, False]:
        for features in feature_combinations:
            try:
                result = choose_parameters_arima(product_id=product_id, df_train=data, feature_set=features,
                                                 target_col=target_col, winsorize=winsorize, seasonal_period=seasonal_period)
                if result:
                    print(f"✅ AutoARIMA Success - Features: {len(features)} "
                          f"RMSE: {result['metrics']['RMSE']:.2f}")
                    results.append(result)
            except Exception as e:
                print(f"⚠️ AutoARIMA failed: {str(e)}")
    
    if not results:
        raise ValueError(f"No valid ARIMA configurations for {product_id}")
    
    # Select best configuration
    best_config = min(results, key=lambda x: x['metrics']['RMSE'])
    
    print(f"\n✅ Best ARIMA configuration for {product_id}:")
    print(f"- Method: {best_config['method']}")
    print(f"- Winsorize: {best_config['winsorize']}")
    print(f"- Features: {best_config['features']}")
    print(f"- Order: {best_config['model_order']}")
    print(f"- Seasonal Order: {best_config['seasonal_order']}")
    print(f"- RMSE: {best_config['metrics']['RMSE']:.2f}")
    
    return best_config, results

##### <span style="background-color:#000027; padding:5px; border-radius:5px;">**3.6. Results Comparison**</span> <a id='compare'></a>  

Click [here](#table-of-contents) ⬆️ to return to the Index.

In [60]:
display_excel_sheets_side_by_side("Modeling_results.xlsx")

Model,product_id,winsorize,features,RMSE,Overfit_Score,Parameters,Feature Importance
TimeGPT,P1,True,"PRO27826_org, MAB_ELE_SHP1100",3475393.6312,-0.3474,NaN,NaN
Prophet,P1,True,"['MAB_ELE_SHP840', 'PRO27826_org', 'MAB_ELE_SHP1100']",2164903.6986,-0.5316,NaN,NaN
ARIMA,P1,True,"['PRI27276_org', 'PRO27826_org', 'MAB_ELE_PRO276', 'MAB_ELE_SHP1100']",2077977.6128,-0.9453,"arima: auto; model_order: (1, 0, 0); seasonal_order: (0, 0, 0, 0)",NaN
XGBoost,P1,True,"['P1_lag_6', 'P1_lag_18', 'P1_ma_6', 'P1_ma_18']",1756694.2558,22.3548,"{'colsample_bytree': 1.0, 'learning_rate': 0.1, 'max_depth': 3, 'min_child_weight': 1, 'n_estimators': 100, 'subsample': 0.9}","{'P1_lag_6': 0.30282173, 'P1_lag_18': 0.18046999, 'P1_ma_6': 0.30943987, 'P1_ma_18': 0.20726843}"
Model,product_id,winsorize,features,RMSE,Overfit_Score,Parameters,Feature Importance
TimeGPT,P3,True,PRO27826_org,1400586.7625,-0.5273,NaN,NaN
Prophet,P3,True,['PRI27840_org'],4038685.9300,5.8188,NaN,NaN
ARIMA,P3,True,['PRI27840_org'],2177873.1620,-0.3089,"arima: auto; model_order: (1, 0, 1); seasonal_order: (0, 1, 1, 12)",NaN
XGBoost,P3,True,"['P3_lag_12', 'P3_ma_12']",1142677.4785,-0.1396,"{'colsample_bytree': 1.0, 'learning_rate': 0.1, 'max_depth': 2, 'min_child_weight': 3, 'n_estimators': 50, 'subsample': 0.9}","{'P3_lag_12': 0.5673661, 'P3_ma_12': 0.43263388}"
Model,product_id,winsorize,features,RMSE,Overfit_Score,Parameters,Feature Importance


#### <span style="background-color:#000027; padding:5px; border-radius:5px;">**4. Exogenous Variables Prediction**</span> <a id='market'></a>  

Click [here](#table-of-contents) ⬆️ to return to the Index.


#### Prediction for he Macro Features used

In [ ]:
# Define the function to classify variables
def classify_variable(series):
    """Classifies variable based on normality and stationarity tests."""
    
    # Remove NaN values for testing
    clean_series = series.dropna()

    # Check for normality
    if len(clean_series) > 3:
        stat, p_value = shapiro(clean_series)
        is_normal = p_value > 0.05  # p-value > 0.05 means normal
    else:
        is_normal = False  # Not enough data to test normality

    # Check for stationarity
    if len(clean_series) > 3:
        adf_stat, adf_p_value, _, _, _, _ = adfuller(clean_series)
        is_stationary = adf_p_value < 0.05  # p-value < 0.05 means stationary
    else:
        is_stationary = False  # Not enough data

    return is_normal, is_stationary

# Function to automatically fill missing values
def auto_impute_missing_values(df_train, df_test):
    """Automatically selects the best imputation method for each missing variable."""
    
    # Identify missing columns in test set
    missing_columns = df_test.columns[df_test.isnull().any()]
    
    # Iterate through missing columns
    for col in missing_columns:
        print(f"Processing: {col}")

        series = df_train[col]  # Use train data for imputation
        is_normal, is_stationary = classify_variable(series)

        if is_normal:
            # Case 1: Normally distributed → Sample from normal distribution
            print(f" - {col} is normal → Using Mean & Std Sampling")
            mean_value, std_value = series.mean(), series.std()
            num_missing = df_test[col].isnull().sum()
            predictions = norm.rvs(loc=mean_value, scale=std_value, size=num_missing)
        
        elif is_stationary:
            # Case 2: Stationary but non-normal → Simple Exponential Smoothing
            print(f" - {col} is stationary → Using Simple Exponential Smoothing")
            model = SimpleExpSmoothing(series.dropna()).fit()
            predictions = model.forecast(steps=df_test[col].isnull().sum())

        elif not is_stationary:
            # Case 3: Non-Stationary → ARIMA
            print(f" - {col} is non-stationary → Using ARIMA")
            model = ARIMA(series.dropna(), order=(1, 1, 1))  # (p,d,q) chosen based on domain knowledge
            fitted_model = model.fit()
            predictions = fitted_model.forecast(steps=df_test[col].isnull().sum())

        else:
            # Case 4: If nothing works → Use XGBoost Regression
            print(f" - {col} is complex → Using XGBoost Regression")
            train_data = df_train.dropna(subset=[col])  # Drop missing values for training
            X_train = train_data.drop(columns=[col])  # Exclude target column
            y_train = train_data[col]  # Target column

            X_test = df_test.loc[df_test[col].isnull(), X_train.columns]  # Only missing values

            model = XGBRegressor(n_estimators=100, learning_rate=0.1)
            model.fit(X_train, y_train)
            predictions = model.predict(X_test)

        # Assign predictions
        missing_indexes = df_test[df_test[col].isnull()].index
        df_test.loc[missing_indexes, col] = predictions

    return df_test

# Example usage
df_train = remerged_data[1]  # Use remerged train data
df_test = test_1.copy()  # Copy test set

# Apply automatic imputation
df_test_filled = auto_impute_missing_values(df_train, df_test)

# Check results
print(df_test_filled.head())